# Feature engineering

In [39]:
#Import all neccessary libraries
import os
import bson
import numpy as np
import pandas as pd; pd.set_option('display.max_columns', None)

In [40]:
os.chdir('c:\\Users\\kavou\\OneDrive\\Έγγραφα\\Πρακτική - Μεταπτυχιακό\\MSA\\Courses\\CSE 6748\\LBA project\\Data\\pickle')
os.getcwd()

'c:\\Users\\kavou\\OneDrive\\Έγγραφα\\Πρακτική - Μεταπτυχιακό\\MSA\\Courses\\CSE 6748\\LBA project\\Data\\pickle'

In [41]:
#Import neccessary pickle files
games = pd.read_pickle(r'NCAAM1 22_23 gamesdf.pickle')
shots = pd.read_pickle(r'Cleaned NCAAM1 22_23 shotsdf.pickle')
player = pd.read_pickle(r'NCAAM1 22_23 playerdictionary.pickle')
segment = pd.DataFrame.from_records(pd.read_pickle(r'NCAAM1 22_23 segment data.pickle'))
Last_season_2P_features_df = pd.read_pickle(r'Last_season_2P_features_df.pickle')
Last_season_3P_features_df = pd.read_pickle(r'Last_season_3P_features_df.pickle')

In [42]:
segment = segment[['game_id','players_team1','players_team2']]
segment["all_players"] = segment["players_team1"]+segment["players_team2"]
segments_df_long = segment.explode(["all_players"])
segments_df_long = segments_df_long.drop(columns=['players_team1','players_team2'])
segments_df_long = segments_df_long.drop_duplicates()
segments_df_long

,game_id,all_players
0,636a5fc5531336e79c342993,618d6f44ed05764db467fe48
0,636a5fc5531336e79c342993,636a5d88531336e79c342588
0,636a5fc5531336e79c342993,636a5d88531336e79c342589
0,636a5fc5531336e79c342993,636a5d88531336e79c34258a
0,636a5fc5531336e79c342993,618daeb7191f8b652ceb990b
...,...,...
203680,648b3a9e98debb5769861e35,618ff0bcd1901c63278a0c7b
203681,648b3a9e98debb5769861e35,636d88521f186408f6c60d65
203682,648b3a9e98debb5769861e35,636d88521f186408f6c60d66
203682,648b3a9e98debb5769861e35,636d86121f186408f6c56e37


In [43]:
segments_df_long['played']=1
segments_df_long

,game_id,all_players,played
0,636a5fc5531336e79c342993,618d6f44ed05764db467fe48,1
0,636a5fc5531336e79c342993,636a5d88531336e79c342588,1
0,636a5fc5531336e79c342993,636a5d88531336e79c342589,1
0,636a5fc5531336e79c342993,636a5d88531336e79c34258a,1
0,636a5fc5531336e79c342993,618daeb7191f8b652ceb990b,1
...,...,...,...
203680,648b3a9e98debb5769861e35,618ff0bcd1901c63278a0c7b,1
203681,648b3a9e98debb5769861e35,636d88521f186408f6c60d65,1
203682,648b3a9e98debb5769861e35,636d88521f186408f6c60d66,1
203682,648b3a9e98debb5769861e35,636d86121f186408f6c56e37,1


Create a dataframe with all possible combinations of team_id, game_id, player_id

In [44]:
#First we will melt the games dataframe two combine the team 1 and and team 2 columns into one
games_df = pd.melt(games, id_vars=['game_id','season','league','date','has_locations','has_segments','hometeam'], 
                   value_vars=['team1', 'team2'], value_name='team_id').drop(columns=['variable'])
#Next, we will sort the dataframe from newest to oldest date
games_df.date = games_df.date.apply(pd.to_datetime)
games_df = games_df.sort_values(['date'], ascending=False)
games_df

,game_id,season,league,date,has_locations,has_segments,hometeam,team_id
6089,642dda80e85b0cd830c4321c,22-23,NCAAM1+,2023-04-03,True,True,None,5e875e684dc25ebee5af6aaf
12212,642dda80e85b0cd830c4321c,22-23,NCAAM1+,2023-04-03,True,True,None,5e875e684dc25ebee5af6773
12207,642aede8dc6a2cc098a967d4,22-23,NCAAM1+,2023-04-01,True,True,None,5e875e684dc25ebee5af6aaf
6085,642aee19dc6a2cc098a96cee,22-23,NCAAM1+,2023-04-01,True,True,None,5e875e684dc25ebee5af680b
6084,642aede8dc6a2cc098a967d4,22-23,NCAAM1+,2023-04-01,True,True,None,5e875e684dc25ebee5af6967
...,...,...,...,...,...,...,...,...
6195,636d895b1f186408f6c63911,22-23,NCAAM1+,2022-11-07,True,True,5e875e684dc25ebee5af6b1f,618aebdf61309f61a1a1dde5
6196,636d896e1f186408f6c63f2f,22-23,NCAAM1+,2022-11-07,True,True,5e875e684dc25ebee5af677f,618ae775a2f85331704507aa
6197,636d89801f186408f6c645b4,22-23,NCAAM1+,2022-11-07,True,True,5e875e684dc25ebee5af6abf,5e875e684dc25ebee5af6abf
6198,636d89931f186408f6c64cf1,22-23,NCAAM1+,2022-11-07,True,True,5e875e684dc25ebee5af69ab,5e875e684dc25ebee5af69ab


In [45]:
#create a new column to track the number of games each team has played since the current one
games_df['games played since'] = games_df.groupby(games_df.team_id).cumcount()
games_df

,game_id,season,league,date,has_locations,has_segments,hometeam,team_id,games played since
6089,642dda80e85b0cd830c4321c,22-23,NCAAM1+,2023-04-03,True,True,None,5e875e684dc25ebee5af6aaf,0
12212,642dda80e85b0cd830c4321c,22-23,NCAAM1+,2023-04-03,True,True,None,5e875e684dc25ebee5af6773,0
12207,642aede8dc6a2cc098a967d4,22-23,NCAAM1+,2023-04-01,True,True,None,5e875e684dc25ebee5af6aaf,1
6085,642aee19dc6a2cc098a96cee,22-23,NCAAM1+,2023-04-01,True,True,None,5e875e684dc25ebee5af680b,0
6084,642aede8dc6a2cc098a967d4,22-23,NCAAM1+,2023-04-01,True,True,None,5e875e684dc25ebee5af6967,0
...,...,...,...,...,...,...,...,...,...
6195,636d895b1f186408f6c63911,22-23,NCAAM1+,2022-11-07,True,True,5e875e684dc25ebee5af6b1f,618aebdf61309f61a1a1dde5,31
6196,636d896e1f186408f6c63f2f,22-23,NCAAM1+,2022-11-07,True,True,5e875e684dc25ebee5af677f,618ae775a2f85331704507aa,0
6197,636d89801f186408f6c645b4,22-23,NCAAM1+,2022-11-07,True,True,5e875e684dc25ebee5af6abf,5e875e684dc25ebee5af6abf,30
6198,636d89931f186408f6c64cf1,22-23,NCAAM1+,2022-11-07,True,True,5e875e684dc25ebee5af69ab,5e875e684dc25ebee5af69ab,34


In [46]:
#merge the players with the shots dataframe to add the team_id column 
#that is neccessary for merging with the games dataframe
player_team = pd.merge(player.rename(columns={'seasonid':'season_id'}), shots[['team_id','season_id']].drop_duplicates(),
                        how='left', on='season_id')
player_team

,player,height,LBA_position,Genius_position,year,season_id,team_id
0,618ff2add1901c63278a2703,78,Mid,F,Sr,63502c8a4e970ea2c1102c04,5e875e684dc25ebee5af6b87
1,618ff2add1901c63278a2704,81,Big,F,So,63502c8a4e970ea2c1102c04,5e875e684dc25ebee5af6b87
2,618ff2add1901c63278a2709,81,Big,F,Sr,63502c8a4e970ea2c1102c04,5e875e684dc25ebee5af6b87
3,618ff2add1901c63278a270a,81,Big,F,Sr,63502c8a4e970ea2c1102c04,5e875e684dc25ebee5af6b87
4,618ff2add1901c63278a270b,81,Big,F,Jr,63502c8a4e970ea2c1102c04,5e875e684dc25ebee5af6b87
...,...,...,...,...,...,...,...
4699,625648232c06a1d63387153e,74,Small,G,So,63502c804e970ea2c1102b18,5e875e684dc25ebee5af67d3
4700,636d8a9d1f186408f6c6a282,82,Big,F,Fr,63502c804e970ea2c1102b18,5e875e684dc25ebee5af67d3
4701,636d8a9d1f186408f6c6a283,77,Mid,G,Fr,63502c804e970ea2c1102b18,5e875e684dc25ebee5af67d3
4702,637866864650d03587041f9a,74,Small,G,Fr,63502c804e970ea2c1102b18,5e875e684dc25ebee5af67d3


In [47]:
#Finally create the dataframe with all the neccessary columns 
team_player_game_df = pd.merge(player_team[['team_id','player']], games_df[['team_id','game_id','games played since']], 
                               on='team_id').rename(columns={'player':'player_id'})
team_player_game_df.to_pickle("team_player_game_df.pickle") 
team_player_game_df

,team_id,player_id,game_id,games played since
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4
...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32


In [48]:
#Find the maximum number of games for one team (will be used later)
most_team_games = team_player_game_df['games played since'].max()+1
most_team_games

39

In [49]:
team_player_game_df = pd.merge(team_player_game_df, segments_df_long.rename(columns={'all_players':'player_id'}), on=['game_id','player_id'], how='left')
team_player_game_df.played = team_player_game_df.played.fillna(0).astype(int)
team_player_game_df

,team_id,player_id,game_id,games played since,played
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1,0
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2,0
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3,0
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4,1
...,...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29,0
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30,0
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31,0
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32,0


In [50]:
#Split the Stat column into 2s/3s and Makes/Misses
shots_df = shots.copy()
shots_df.insert(13, 'Type', shots_df.Stat.str[:-4])
shots_df.insert(14, 'Result', shots_df.Stat.str[-4:])
shots_df.Result = np.where(shots_df.Result=='Make', 1, 0)
shots_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back
0,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c3425dd,3,2,1,636a5f91531336e79c3425b4,18:50,1130,ThreeMake,7-1,Three,1,5.0,20.7,37.7,25.26,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,636a5d88531336e79c342589,636a5fa6531336e79c3425e1,3,2,1,636a5f91531336e79c3425b4,18:11,1091,TwoMiss,4-2,Two,0,21.0,20.7,75.5,15.94,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,636a5d88531336e79c34258a,636a5fa6531336e79c3425e3,3,2,1,636a5f91531336e79c3425b4,18:6,1086,TwoMiss,3-1,Two,0,14.0,11.3,28.7,12.54,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f45ed05764db467fe4e,636a5fa6531336e79c3425e8,3,4,1,636a5f91531336e79c3425b5,17:7,1027,TwoMiss,3-2,Two,0,25.0,16.9,90.0,11.67,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False
4,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,636a5d88531336e79c342588,636a5fa6531336e79c3425ea,3,4,1,636a5f91531336e79c3425b5,16:28,988,TwoMiss,4-2,Two,0,25.0,24.4,90.0,19.19,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699561,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab19a32a50e3d7afdc2,648b3a7b98debb5769861cdd,83,85,3,648b3a0b98debb576986198e,1:31,91,TwoMiss,1,Two,0,27.0,7.5,131.4,3.03,False,False,True,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False
699562,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab09a32a50e3d7afdbb,648b3a7b98debb5769861ce0,83,87,3,648b3a0b98debb576986198e,0:37,37,TwoMiss,4-2,Two,0,26.0,20.7,93.7,15.46,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
699563,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab19a32a50e3d7afdc6,648b3a7b98debb5769861ce2,85,87,3,648b3a0b98debb576986198e,0:35,35,TwoMake,1,Two,1,25.5,7.5,102.4,2.32,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False
699564,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab19a32a50e3d7afdc2,648b3a7b98debb5769861ce4,85,88,3,648b3a0b98debb576986198e,0:11,11,TwoMiss,1,Two,0,24.0,9.4,76.5,4.27,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False


In [51]:
#Column D1 is a binary True/False variable that keeps track of whether or not that takes the shots plays for a D1 team
player['D1'] = True
shots_df = pd.merge(shots_df, player[['player', 'D1']].rename(columns={'player':'player_id'}), on='player_id', how='left')
shots_df['D1'] = shots_df['D1'].fillna(False)
shots_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1
0,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c3425dd,3,2,1,636a5f91531336e79c3425b4,18:50,1130,ThreeMake,7-1,Three,1,5.0,20.7,37.7,25.26,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,636a5d88531336e79c342589,636a5fa6531336e79c3425e1,3,2,1,636a5f91531336e79c3425b4,18:11,1091,TwoMiss,4-2,Two,0,21.0,20.7,75.5,15.94,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
2,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,636a5d88531336e79c34258a,636a5fa6531336e79c3425e3,3,2,1,636a5f91531336e79c3425b4,18:6,1086,TwoMiss,3-1,Two,0,14.0,11.3,28.7,12.54,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f45ed05764db467fe4e,636a5fa6531336e79c3425e8,3,4,1,636a5f91531336e79c3425b5,17:7,1027,TwoMiss,3-2,Two,0,25.0,16.9,90.0,11.67,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,True
4,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,636a5d88531336e79c342588,636a5fa6531336e79c3425ea,3,4,1,636a5f91531336e79c3425b5,16:28,988,TwoMiss,4-2,Two,0,25.0,24.4,90.0,19.19,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698294,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab19a32a50e3d7afdc2,648b3a7b98debb5769861cdd,83,85,3,648b3a0b98debb576986198e,1:31,91,TwoMiss,1,Two,0,27.0,7.5,131.4,3.03,False,False,True,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,True
698295,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab09a32a50e3d7afdbb,648b3a7b98debb5769861ce0,83,87,3,648b3a0b98debb576986198e,0:37,37,TwoMiss,4-2,Two,0,26.0,20.7,93.7,15.46,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
698296,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab19a32a50e3d7afdc6,648b3a7b98debb5769861ce2,85,87,3,648b3a0b98debb576986198e,0:35,35,TwoMake,1,Two,1,25.5,7.5,102.4,2.32,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True
698297,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab19a32a50e3d7afdc2,648b3a7b98debb5769861ce4,85,88,3,648b3a0b98debb576986198e,0:11,11,TwoMiss,1,Two,0,24.0,9.4,76.5,4.27,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False,False,False,False,False,True


Feature creation for 2 pointers

In [52]:
#Filter on 2 point shots
Twos_df = shots_df[shots_df.Type=='Two']
two_pointers_df = Twos_df.groupby(['player_id','game_id']).Result.agg(
    ['sum','count']).reset_index().rename(columns={'sum':'2PM', 'count':'2PA'})
two_pointers_df

,player_id,game_id,2PM,2PA
0,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,2,2
1,618bf7f3815098ca6660f334,636f9e5827131d1edcad44ed,1,5
2,618bf7f3815098ca6660f334,637310726c21c0a3cfa28625,1,4
3,618bf7f3815098ca6660f334,6378674f4650d0358704664d,2,5
4,618bf7f3815098ca6660f334,637c5431fdedb41779d81fcc,5,7
...,...,...,...,...
98600,640004f5effffd352c7ea690,640004fbeffffd352c7eab2e,1,3
98601,640004f5effffd352c7ea691,640004fbeffffd352c7eab2e,0,1
98602,640026c5af005c7ca66d28f7,640026cbaf005c7ca66d2d50,0,1
98603,64018832f25daee41501b665,64018839f25daee41501ba68,0,1


In [53]:
#merge with the dataframe containing the team_id, game_id, player_id combinations
two_pointers_df = pd.merge(team_player_game_df, two_pointers_df, how= 'left', on=['player_id','game_id']).fillna(0)
two_pointers_df

,team_id,player_id,game_id,games played since,played,2PM,2PA
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0,0.0,0.0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1,0,0.0,0.0
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2,0,0.0,0.0
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3,0,0.0,0.0
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4,1,0.0,0.0
...,...,...,...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29,0,0.0,0.0
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30,0,0.0,0.0
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31,0,0.0,0.0
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32,0,0.0,0.0


In [54]:
#Create made and attempted game totals for each of the following shot types
for stat in ['Dunk', 'Layup', 'Floater', 'Hook Shot', 'Jumpshot', 'Pullup', 'Turnaround', 'Step Back']:
    temp = Twos_df[Twos_df[stat]==True]
    temp_df = temp.groupby(['player_id','game_id']).Result.agg(['sum','count']).reset_index().rename(
        columns={'sum': stat+'s made', 'count': stat+'s attempted'})
    two_pointers_df = pd.merge(two_pointers_df, temp_df, how= 'left', on=['player_id','game_id']).fillna(0)
two_pointers_df

,team_id,player_id,game_id,games played since,played,2PM,2PA,Dunks made,Dunks attempted,Layups made,Layups attempted,Floaters made,Floaters attempted,Hook Shots made,Hook Shots attempted,Jumpshots made,Jumpshots attempted,Pullups made,Pullups attempted,Turnarounds made,Turnarounds attempted,Step Backs made,Step Backs attempted
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
#Calculated percentage of made 2p jumpshots that were assisted
temp_df = Twos_df[(Twos_df['Jumpshot']==True)&(Twos_df['Result']==1)].groupby(
    ['player_id','game_id'])['Assisted'].sum().reset_index().rename(columns={'Assisted': 'Jumpshots assisted on'})
temp_df

,player_id,game_id,Jumpshots assisted on
0,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,1
1,618bf7f3815098ca6660f334,637310726c21c0a3cfa28625,1
2,618bf7f3815098ca6660f334,6378674f4650d0358704664d,0
3,618bf7f3815098ca6660f334,637c5431fdedb41779d81fcc,3
4,618bf7f3815098ca6660f334,637effe51280212ca084ca78,1
...,...,...,...
22923,63f58beb42b88679deff8865,63f58bf142b88679deff8c75,0
22924,63f58beb42b88679deff8866,63f58bf142b88679deff8c75,0
22925,640004f4effffd352c7ea688,640004fbeffffd352c7eab2e,1
22926,640004f5effffd352c7ea68c,640004fbeffffd352c7eab2e,0


In [56]:
two_pointers_df = pd.merge(two_pointers_df, temp_df, how= 'left', on=['player_id','game_id']).fillna(0)
two_pointers_df

,team_id,player_id,game_id,games played since,played,2PM,2PA,Dunks made,Dunks attempted,Layups made,Layups attempted,Floaters made,Floaters attempted,Hook Shots made,Hook Shots attempted,Jumpshots made,Jumpshots attempted,Pullups made,Pullups attempted,Turnarounds made,Turnarounds attempted,Step Backs made,Step Backs attempted,Jumpshots assisted on
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
#2P% per zone for each player
Zones2 = Twos_df.groupby(['player_id','game_id','Zone']).Result.agg(['sum','count']).reset_index()
Zones2

,player_id,game_id,Zone,sum,count
0,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,4-1,1,1
1,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,4-2,1,1
2,618bf7f3815098ca6660f334,636f9e5827131d1edcad44ed,2-2,1,2
3,618bf7f3815098ca6660f334,636f9e5827131d1edcad44ed,4-2,0,2
4,618bf7f3815098ca6660f334,636f9e5827131d1edcad44ed,4-3,0,1
...,...,...,...,...,...
237764,640004f5effffd352c7ea690,640004fbeffffd352c7eab2e,3-1,0,1
237765,640004f5effffd352c7ea691,640004fbeffffd352c7eab2e,4-1,0,1
237766,640026c5af005c7ca66d28f7,640026cbaf005c7ca66d2d50,4-2,0,1
237767,64018832f25daee41501b665,64018839f25daee41501ba68,3-2,0,1


In [58]:
#Pivot on Zone column before merging with the features dataframe
pivot_zones = Zones2.pivot(index=['player_id','game_id'], columns='Zone')[['sum','count']].reset_index().rename(
    columns={'sum':'made','count':'attempted'})
pivot_zones.columns = pivot_zones.columns.map(lambda x: f'{x[1]} {x[0]}')#merge the two index level names
pivot_zones.columns = pivot_zones.columns.map(lambda x: x[1:])#remove leading space
pivot_zones

,player_id,game_id,1 made,2-1 made,2-2 made,2-3 made,3-1 made,3-2 made,3-3 made,4-1 made,4-2 made,4-3 made,1 attempted,2-1 attempted,2-2 attempted,2-3 attempted,3-1 attempted,3-2 attempted,3-3 attempted,4-1 attempted,4-2 attempted,4-3 attempted
0,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
1,618bf7f3815098ca6660f334,636f9e5827131d1edcad44ed,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0
2,618bf7f3815098ca6660f334,637310726c21c0a3cfa28625,0.0,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
3,618bf7f3815098ca6660f334,6378674f4650d0358704664d,1.0,NaN,NaN,0.0,NaN,0.0,0.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN
4,618bf7f3815098ca6660f334,637c5431fdedb41779d81fcc,1.0,NaN,1.0,NaN,0.0,NaN,NaN,NaN,3.0,NaN,2.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98600,640004f5effffd352c7ea690,640004fbeffffd352c7eab2e,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
98601,640004f5effffd352c7ea691,640004fbeffffd352c7eab2e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
98602,640026c5af005c7ca66d28f7,640026cbaf005c7ca66d2d50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
98603,64018832f25daee41501b665,64018839f25daee41501ba68,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [59]:
two_pointers_df = pd.merge(two_pointers_df, pivot_zones, how= 'left', on=['player_id','game_id']).fillna(0)
two_pointers_df

,team_id,player_id,game_id,games played since,played,2PM,2PA,Dunks made,Dunks attempted,Layups made,Layups attempted,Floaters made,Floaters attempted,Hook Shots made,Hook Shots attempted,Jumpshots made,Jumpshots attempted,Pullups made,Pullups attempted,Turnarounds made,Turnarounds attempted,Step Backs made,Step Backs attempted,Jumpshots assisted on,1 made,2-1 made,2-2 made,2-3 made,3-1 made,3-2 made,3-3 made,4-1 made,4-2 made,4-3 made,1 attempted,2-1 attempted,2-2 attempted,2-3 attempted,3-1 attempted,3-2 attempted,3-3 attempted,4-1 attempted,4-2 attempted,4-3 attempted
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
two_pointers_df.columns[4:].to_list()

['played',
 '2PM',
 '2PA',
 'Dunks made',
 'Dunks attempted',
 'Layups made',
 'Layups attempted',
 'Floaters made',
 'Floaters attempted',
 'Hook Shots made',
 'Hook Shots attempted',
 'Jumpshots made',
 'Jumpshots attempted',
 'Pullups made',
 'Pullups attempted',
 'Turnarounds made',
 'Turnarounds attempted',
 'Step Backs made',
 'Step Backs attempted',
 'Jumpshots assisted on',
 '1 made',
 '2-1 made',
 '2-2 made',
 '2-3 made',
 '3-1 made',
 '3-2 made',
 '3-3 made',
 '4-1 made',
 '4-2 made',
 '4-3 made',
 '1 attempted',
 '2-1 attempted',
 '2-2 attempted',
 '2-3 attempted',
 '3-1 attempted',
 '3-2 attempted',
 '3-3 attempted',
 '4-1 attempted',
 '4-2 attempted',
 '4-3 attempted']

In [61]:
#Create a new dataframe for the last 1, 3, 5, 10 games, plus the season total
#Use the rolling function to compute the last x games sums, NOT INCLUDING the game in each row (closed='left')
Last_game_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(closed='left',window=1).sum()
Last_3_games_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(closed='left',window=3).sum()
Last_5_games_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(closed='left',window=5).sum()
Last_10_games_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(closed='left',window=10).sum()
Season_2P = two_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[two_pointers_df.columns[4:].to_list()].rolling(
        closed='left',window=most_team_games, min_periods=1).sum()

In [62]:
#For each dataframe and each shot/zone type compute the respective percentages
for df in [Last_game_2P, Last_3_games_2P, Last_5_games_2P, Last_10_games_2P, Season_2P]:
    df.insert(5, '2P%', df['2PM']/df['2PA'])
    df.insert(8, 'Dunk FG%', df['Dunks made']/df['Dunks attempted'])
    df.insert(11, 'Layup FG%', df['Layups made']/df['Layups attempted'])
    df.insert(14, 'Floater FG%', df['Floaters made']/df['Floaters attempted'])
    df.insert(17, 'Hook Shot FG%', df['Hook Shots made']/df['Hook Shots attempted'])
    df.insert(20, 'Jumpshot FG%', df['Jumpshots made']/df['Jumpshots attempted'])
    df.insert(23, 'Pullup FG%', df['Pullups made']/df['Pullups attempted'])
    df.insert(26, 'Turnaround FG%', df['Turnarounds made']/df['Turnarounds attempted'])
    df.insert(29, 'Step Back FG%', df['Step Backs made']/df['Step Backs attempted'])
    df.insert(31, 'Jumpshots assisted%', df['Jumpshots assisted on']/df['2PM'])
    df['zone 1 FG%'] = df['1 made']/df['1 attempted']
    df['zone 2-1 FG%'] = df['2-1 made']/df['2-1 attempted']
    df['zone 2-2 FG%'] = df['2-2 made']/df['2-2 attempted']
    df['zone 2-3 FG%'] = df['2-3 made']/df['2-3 attempted']
    df['zone 3-1 FG%'] = df['3-1 made']/df['3-1 attempted']
    df['zone 3-2 FG%'] = df['3-2 made']/df['3-2 attempted']
    df['zone 3-3 FG%'] = df['3-3 made']/df['3-3 attempted']
    df['zone 4-1 FG%'] = df['4-1 made']/df['4-1 attempted']
    df['zone 4-2 FG%'] = df['4-2 made']/df['4-2 attempted']
    df['zone 4-3 FG%'] = df['4-3 made']/df['4-3 attempted']

In [63]:
#For each dataframe and each shot/zone type normalize the respective makes and attempts
for df in [Last_game_2P, Last_3_games_2P, Last_5_games_2P, Last_10_games_2P, Season_2P]:
    df['2PM'] = np.where(df.played.isna()==True, df['2PM'], df['2PM']/df.played)
    df['2PA'] = np.where(df.played.isna()==True, df['2PA'], df['2PA']/df.played)
    df['Dunks made'] = np.where(df.played.isna()==True, df['Dunks made'], df['Dunks made']/df.played)
    df['Dunks attempted'] = np.where(df.played.isna()==True, df['Dunks attempted'], df['Dunks attempted']/df.played)
    df['Layups made'] = np.where(df.played.isna()==True, df['Layups made'], df['Layups made']/df.played)
    df['Layups attempted'] = np.where(df.played.isna()==True, df['Layups attempted'], df['Layups attempted']/df.played)
    df['Floaters made'] = np.where(df.played.isna()==True, df['Floaters made'], df['Floaters made']/df.played)
    df['Floaters attempted'] = np.where(df.played.isna()==True, df['Floaters attempted'], df['Floaters attempted']/df.played)
    df['Hook Shots made'] = np.where(df.played.isna()==True, df['Hook Shots made'], df['Hook Shots made']/df.played)
    df['Hook Shots attempted'] = np.where(df.played.isna()==True, df['Hook Shots attempted'], df['Hook Shots attempted']/df.played)
    df['Jumpshots made'] = np.where(df.played.isna()==True, df['Jumpshots made'], df['Jumpshots made']/df.played)
    df['Jumpshots attempted'] = np.where(df.played.isna()==True, df['Jumpshots attempted'], df['Jumpshots attempted']/df.played)
    df['Pullups made'] = np.where(df.played.isna()==True, df['Pullups made'], df['Pullups made']/df.played)
    df['Pullups attempted'] = np.where(df.played.isna()==True, df['Pullups attempted'], df['Pullups attempted']/df.played)
    df['Turnarounds made'] = np.where(df.played.isna()==True, df['Turnarounds made'], df['Turnarounds made']/df.played)
    df['Turnarounds attempted'] = np.where(df.played.isna()==True, df['Turnarounds attempted'], df['Turnarounds attempted']/df.played)
    df['Step Backs made'] = np.where(df.played.isna()==True, df['Step Backs made'], df['Step Backs made']/df.played)
    df['Step Backs attempted'] = np.where(df.played.isna()==True, df['Step Backs attempted'], df['Step Backs attempted']/df.played)
    df['1 made'] = np.where(df.played.isna()==True, df['1 made'], df['1 made']/df.played)
    df['1 attempted'] = np.where(df.played.isna()==True, df['1 attempted'], df['1 attempted']/df.played)
    df['2-1 made'] = np.where(df.played.isna()==True, df['2-1 made'], df['2-1 made']/df.played)
    df['2-1 attempted'] = np.where(df.played.isna()==True, df['2-1 attempted'], df['2-1 attempted']/df.played)
    df['2-2 made'] = np.where(df.played.isna()==True, df['2-2 made'], df['2-2 made']/df.played)
    df['2-2 attempted'] = np.where(df.played.isna()==True, df['2-2 attempted'], df['2-2 attempted']/df.played)
    df['2-3 made'] = np.where(df.played.isna()==True, df['2-3 made'], df['2-3 made']/df.played)
    df['2-3 attempted'] = np.where(df.played.isna()==True, df['2-3 attempted'], df['2-3 attempted']/df.played)
    df['3-1 made'] = np.where(df.played.isna()==True, df['3-1 made'], df['3-1 made']/df.played)
    df['3-1 attempted'] = np.where(df.played.isna()==True, df['3-1 attempted'], df['3-1 attempted']/df.played)
    df['3-2 made'] = np.where(df.played.isna()==True, df['3-2 made'], df['3-2 made']/df.played)
    df['3-2 attempted'] = np.where(df.played.isna()==True, df['3-2 attempted'], df['3-2 attempted']/df.played)
    df['3-3 made'] = np.where(df.played.isna()==True, df['3-3 made'], df['3-3 made']/df.played)
    df['3-3 attempted'] = np.where(df.played.isna()==True, df['3-3 attempted'], df['3-3 attempted']/df.played)
    df['4-1 made'] = np.where(df.played.isna()==True, df['4-1 made'], df['4-1 made']/df.played)
    df['4-1 attempted'] = np.where(df.played.isna()==True, df['4-1 attempted'], df['4-1 attempted']/df.played)
    df['4-2 made'] = np.where(df.played.isna()==True, df['4-2 made'], df['4-2 made']/df.played)
    df['4-2 attempted'] = np.where(df.played.isna()==True, df['4-2 attempted'], df['4-2 attempted']/df.played)
    df['4-3 made'] = np.where(df.played.isna()==True, df['4-3 made'], df['4-3 made']/df.played)
    df['4-3 attempted'] = np.where(df.played.isna()==True, df['4-3 attempted'], df['4-3 attempted']/df.played)

In [64]:
#Rename the columns before merging all the dataframes together
Last_game_2P.columns = Last_game_2P.columns.map(lambda x: f'L1 {x}')
Last_3_games_2P.columns = Last_3_games_2P.columns.map(lambda x: f'L3 {x}')
Last_5_games_2P.columns = Last_5_games_2P.columns.map(lambda x: f'L5 {x}')
Last_10_games_2P.columns = Last_10_games_2P.columns.map(lambda x: f'L10 {x}')
Season_2P.columns = Season_2P.columns.map(lambda x: f'TOT {x}')
Last_game_2P = Last_game_2P.rename(columns={'L1 team_id':'team_id','L1 player_id':'player_id'})
Last_3_games_2P = Last_3_games_2P.rename(columns={'L3 team_id':'team_id','L3 player_id':'player_id'})
Last_5_games_2P = Last_5_games_2P.rename(columns={'L5 team_id':'team_id','L5 player_id':'player_id'})
Last_10_games_2P = Last_10_games_2P.rename(columns={'L10 team_id':'team_id','L10 player_id':'player_id'})
Season_2P = Season_2P.rename(columns={'TOT team_id':'team_id','TOT player_id':'player_id'})

In [65]:
#Concatenate all the dataframes into one
feature_df_2P = pd.concat([team_player_game_df, Last_game_2P.iloc[:,2:], 
                           Last_3_games_2P.iloc[:,2:], Last_5_games_2P.iloc[:,2:], 
                           Last_10_games_2P.iloc[:,2:], Season_2P.iloc[:,2:]], axis=1)
feature_df_2P.to_pickle("2P_features_df.pickle")#write the data to a pickle file
feature_df_2P

,team_id,player_id,game_id,games played since,played,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Dunks made,L1 Dunks attempted,L1 Dunk FG%,L1 Layups made,L1 Layups attempted,L1 Layup FG%,L1 Floaters made,L1 Floaters attempted,L1 Floater FG%,L1 Hook Shots made,L1 Hook Shots attempted,L1 Hook Shot FG%,L1 Jumpshots made,L1 Jumpshots attempted,L1 Jumpshot FG%,L1 Pullups made,L1 Pullups attempted,L1 Pullup FG%,L1 Turnarounds made,L1 Turnarounds attempted,L1 Turnaround FG%,L1 Step Backs made,L1 Step Backs attempted,L1 Step Back FG%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L1 1 made,L1 2-1 made,L1 2-2 made,L1 2-3 made,L1 3-1 made,L1 3-2 made,L1 3-3 made,L1 4-1 made,L1 4-2 made,L1 4-3 made,L1 1 attempted,L1 2-1 attempted,L1 2-2 attempted,L1 2-3 attempted,L1 3-1 attempted,L1 3-2 attempted,L1 3-3 attempted,L1 4-1 attempted,L1 4-2 attempted,L1 4-3 attempted,L1 zone 1 FG%,L1 zone 2-1 FG%,L1 zone 2-2 FG%,L1 zone 2-3 FG%,L1 zone 3-1 FG%,L1 zone 3-2 FG%,L1 zone 3-3 FG%,L1 zone 4-1 FG%,L1 zone 4-2 FG%,L1 zone 4-3 FG%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Dunks made,L3 Dunks attempted,L3 Dunk FG%,L3 Layups made,L3 Layups attempted,L3 Layup FG%,L3 Floaters made,L3 Floaters attempted,L3 Floater FG%,L3 Hook Shots made,L3 Hook Shots attempted,L3 Hook Shot FG%,L3 Jumpshots made,L3 Jumpshots attempted,L3 Jumpshot FG%,L3 Pullups made,L3 Pullups attempted,L3 Pullup FG%,L3 Turnarounds made,L3 Turnarounds attempted,L3 Turnaround FG%,L3 Step Backs made,L3 Step Backs attempted,L3 Step Back FG%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L3 1 made,L3 2-1 made,L3 2-2 made,L3 2-3 made,L3 3-1 made,L3 3-2 made,L3 3-3 made,L3 4-1 made,L3 4-2 made,L3 4-3 made,L3 1 attempted,L3 2-1 attempted,L3 2-2 attempted,L3 2-3 attempted,L3 3-1 attempted,L3 3-2 attempted,L3 3-3 attempted,L3 4-1 attempted,L3 4-2 attempted,L3 4-3 attempted,L3 zone 1 FG%,L3 zone 2-1 FG%,L3 zone 2-2 FG%,L3 zone 2-3 FG%,L3 zone 3-1 FG%,L3 zone 3-2 FG%,L3 zone 3-3 FG%,L3 zone 4-1 FG%,L3 zone 4-2 FG%,L3 zone 4-3 FG%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Dunks made,L5 Dunks attempted,L5 Dunk FG%,L5 Layups made,L5 Layups attempted,L5 Layup FG%,L5 Floaters made,L5 Floaters attempted,L5 Floater FG%,L5 Hook Shots made,L5 Hook Shots attempted,L5 Hook Shot FG%,L5 Jumpshots made,L5 Jumpshots attempted,L5 Jumpshot FG%,L5 Pullups made,L5 Pullups attempted,L5 Pullup FG%,L5 Turnarounds made,L5 Turnarounds attempted,L5 Turnaround FG%,L5 Step Backs made,L5 Step Backs attempted,L5 Step Back FG%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L5 1 made,L5 2-1 made,L5 2-2 made,L5 2-3 made,L5 3-1 made,L5 3-2 made,L5 3-3 made,L5 4-1 made,L5 4-2 made,L5 4-3 made,L5 1 attempted,L5 2-1 attempted,L5 2-2 attempted,L5 2-3 attempted,L5 3-1 attempted,L5 3-2 attempted,L5 3-3 attempted,L5 4-1 attempted,L5 4-2 attempted,L5 4-3 attempted,L5 zone 1 FG%,L5 zone 2-1 FG%,L5 zone 2-2 FG%,L5 zone 2-3 FG%,L5 zone 3-1 FG%,L5 zone 3-2 FG%,L5 zone 3-3 FG%,L5 zone 4-1 FG%,L5 zone 4-2 FG%,L5 zone 4-3 FG%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Dunks made,L10 Dunks attempted,L10 Dunk FG%,L10 Layups made,L10 Layups attempted,L10 Layup FG%,L10 Floaters made,L10 Floaters attempted,L10 Floater FG%,L10 Hook Shots made,L10 Hook Shots attempted,L10 Hook Shot FG%,L10 Jumpshots made,L10 Jumpshots attempted,L10 Jumpshot FG%,L10 Pullups made,L10 Pullups attempted,L10 Pullup FG%,L10 Turnarounds made,L10 Turnarounds attempted,L10 Turnaround FG%,L10 Step Backs made,L10 Step Backs attempted,L10 Step Back FG%,L10 Jumpshots assisted on,L10 Jumpshots assisted%,L10 1 made,L10 2-1 made,L10 2-2 made,L10 2-3 made,L10 3-1 made,L10 3-2 made,L10 3-3 made,L10 4-1 made,L10 4-2 made,L10 4-3 made,L10 1 attempted,L10 2-1 attempted,L10 2-2 attempted,L10 2-3 attempted,L10 3-1 attempted,L10 3-2 attempted,L10 3-3 attempted,L10 4-1 attempted,L10 4-2 attempted,L10 4-3 attempted,L10 zone 1 FG%,L10 zone 2-1 FG%,L10 zone 2-2 FG%,L10 zone 2-3 FG%,L10 zone 3-1 FG%,L10 zone 3-2 FG%,L10 zone 3-3 FG%,L10 zone 4-1 FG%,L10 zone 4-2 FG%,L10 zone 4-3 FG%,TOT played,TOT 2PM,TOT 2PA,TOT 2P%,TOT Dunk

In [66]:
#Now that current season's features are created, we will merge them with last season's features
Last_season_2P_features_df

,player_id,Last_season played,Last_season 2PM,Last_season 2PA,Last_season 2P%,Last_season Dunks made,Last_season Dunks attempted,Last_season Dunk FG%,Last_season Layups made,Last_season Layups attempted,Last_season Layup FG%,Last_season Floaters made,Last_season Floaters attempted,Last_season Floater FG%,Last_season Hook Shots made,Last_season Hook Shots attempted,Last_season Hook Shot FG%,Last_season Jumpshots made,Last_season Jumpshots attempted,Last_season Jumpshot FG%,Last_season Pullups made,Last_season Pullups attempted,Last_season Pullup FG%,Last_season Turnarounds made,Last_season Turnarounds attempted,Last_season Turnaround FG%,Last_season Step Backs made,Last_season Step Backs attempted,Last_season Step Back FG%,Last_season Jumpshots assisted on,Last_season Jumpshots assisted%,Last_season 1 made,Last_season 2-1 made,Last_season 2-2 made,Last_season 2-3 made,Last_season 3-1 made,Last_season 3-2 made,Last_season 3-3 made,Last_season 4-1 made,Last_season 4-2 made,Last_season 4-3 made,Last_season 1 attempted,Last_season 2-1 attempted,Last_season 2-2 attempted,Last_season 2-3 attempted,Last_season 3-1 attempted,Last_season 3-2 attempted,Last_season 3-3 attempted,Last_season 4-1 attempted,Last_season 4-2 attempted,Last_season 4-3 attempted,Last_season zone 1 FG%,Last_season zone 2-1 FG%,Last_season zone 2-2 FG%,Last_season zone 2-3 FG%,Last_season zone 3-1 FG%,Last_season zone 3-2 FG%,Last_season zone 3-3 FG%,Last_season zone 4-1 FG%,Last_season zone 4-2 FG%,Last_season zone 4-3 FG%
0,618da14c4db298feb94be0d1,24,2.666667,5.875000,0.453901,0.041667,0.041667,1.0,1.666667,3.250000,0.512821,0.125000,0.291667,0.428571,0.041667,0.083333,0.500000,0.083333,0.750000,0.111111,0.0,0.041667,0.0,0.208333,0.666667,0.3125,0.083333,0.083333,1.0,0.0,0.000000,1.833333,0.166667,0.166667,0.083333,0.083333,0.125000,0.041667,0.041667,0.125000,0.0,3.333333,0.541667,0.625000,0.166667,0.208333,0.458333,0.125000,0.041667,0.375000,0.000000,0.550000,0.307692,0.266667,0.500000,0.400000,0.272727,0.333333,1.0,0.333333,NaN
1,618da14c4db298feb94be0d2,24,3.708333,7.083333,0.523529,0.083333,0.083333,1.0,2.333333,4.500000,0.518519,0.000000,0.041667,0.000000,0.500000,0.833333,0.600000,0.083333,0.583333,0.142857,0.0,0.000000,NaN,0.083333,0.333333,0.2500,0.000000,0.000000,NaN,0.0,0.000000,2.958333,0.166667,0.416667,0.041667,0.041667,0.041667,0.041667,0.000000,0.000000,0.0,4.833333,0.500000,1.041667,0.291667,0.125000,0.166667,0.041667,0.000000,0.041667,0.041667,0.612069,0.333333,0.400000,0.142857,0.333333,0.250000,1.000000,NaN,0.000000,0.0
2,618da14c4db298feb94be0d3,23,0.478261,0.913043,0.523810,0.000000,0.000000,NaN,0.304348,0.521739,0.583333,0.043478,0.043478,1.000000,0.000000,0.043478,0.000000,0.086957,0.130435,0.666667,0.0,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,1.0,0.090909,0.260870,0.086957,0.043478,0.000000,0.000000,0.043478,0.000000,0.000000,0.043478,0.0,0.521739,0.086957,0.086957,0.000000,0.000000,0.130435,0.000000,0.000000,0.086957,0.000000,0.500000,1.000000,0.500000,NaN,NaN,0.333333,NaN,NaN,0.500000,NaN
3,618da14c4db298feb94be0d4,24,2.666667,5.041667,0.528926,0.000000,0.000000,NaN,1.208333,2.375000,0.508772,0.125000,0.166667,0.750000,0.625000,0.916667,0.681818,0.458333,1.000000,0.458333,0.0,0.000000,NaN,0.125000,0.416667,0.3000,0.000000,0.000000,NaN,2.0,0.031250,1.208333,0.333333,0.541667,0.041667,0.041667,0.166667,0.041667,0.041667,0.250000,0.0,2.125000,0.500000,0.875000,0.125000,0.250000,0.583333,0.041667,0.041667,0.500000,0.000000,0.568627,0.666667,0.619048,0.333333,0.166667,0.285714,1.000000,1.0,0.500000,NaN
4,618da14c4db298feb94be0d5,22,1.727273,4.181818,0.413043,0.000000,0.000000,NaN,1.545455,2.818182,0.548387,0.000000,0.136364,0.000000,0.000000,0.090909,0.000000,0.090909,0.818182,0.111111,0.0,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.000000,1.227273,0.090909,0.227273,0.090909,0.000000,0.045455,0.045455,0.000000,0.000000,0.0,2.409091,0.227273,0.545455,0.272727,0.045455,0.318182,0.227273,0.045455,0.090909,0.0

In [67]:
reduced_2P_df = pd.merge(Twos_df, feature_df_2P, how='left', on=['team_id','player_id','game_id'])
reduced_2P_df = pd.merge(reduced_2P_df, Last_season_2P_features_df, how='left', on=['player_id'])
reduced_2P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Dunks made,L1 Dunks attempted,L1 Dunk FG%,L1 Layups made,L1 Layups attempted,L1 Layup FG%,L1 Floaters made,L1 Floaters attempted,L1 Floater FG%,L1 Hook Shots made,L1 Hook Shots attempted,L1 Hook Shot FG%,L1 Jumpshots made,L1 Jumpshots attempted,L1 Jumpshot FG%,L1 Pullups made,L1 Pullups attempted,L1 Pullup FG%,L1 Turnarounds made,L1 Turnarounds attempted,L1 Turnaround FG%,L1 Step Backs made,L1 Step Backs attempted,L1 Step Back FG%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L1 1 made,L1 2-1 made,L1 2-2 made,L1 2-3 made,L1 3-1 made,L1 3-2 made,L1 3-3 made,L1 4-1 made,L1 4-2 made,L1 4-3 made,L1 1 attempted,L1 2-1 attempted,L1 2-2 attempted,L1 2-3 attempted,L1 3-1 attempted,L1 3-2 attempted,L1 3-3 attempted,L1 4-1 attempted,L1 4-2 attempted,L1 4-3 attempted,L1 zone 1 FG%,L1 zone 2-1 FG%,L1 zone 2-2 FG%,L1 zone 2-3 FG%,L1 zone 3-1 FG%,L1 zone 3-2 FG%,L1 zone 3-3 FG%,L1 zone 4-1 FG%,L1 zone 4-2 FG%,L1 zone 4-3 FG%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Dunks made,L3 Dunks attempted,L3 Dunk FG%,L3 Layups made,L3 Layups attempted,L3 Layup FG%,L3 Floaters made,L3 Floaters attempted,L3 Floater FG%,L3 Hook Shots made,L3 Hook Shots attempted,L3 Hook Shot FG%,L3 Jumpshots made,L3 Jumpshots attempted,L3 Jumpshot FG%,L3 Pullups made,L3 Pullups attempted,L3 Pullup FG%,L3 Turnarounds made,L3 Turnarounds attempted,L3 Turnaround FG%,L3 Step Backs made,L3 Step Backs attempted,L3 Step Back FG%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L3 1 made,L3 2-1 made,L3 2-2 made,L3 2-3 made,L3 3-1 made,L3 3-2 made,L3 3-3 made,L3 4-1 made,L3 4-2 made,L3 4-3 made,L3 1 attempted,L3 2-1 attempted,L3 2-2 attempted,L3 2-3 attempted,L3 3-1 attempted,L3 3-2 attempted,L3 3-3 attempted,L3 4-1 attempted,L3 4-2 attempted,L3 4-3 attempted,L3 zone 1 FG%,L3 zone 2-1 FG%,L3 zone 2-2 FG%,L3 zone 2-3 FG%,L3 zone 3-1 FG%,L3 zone 3-2 FG%,L3 zone 3-3 FG%,L3 zone 4-1 FG%,L3 zone 4-2 FG%,L3 zone 4-3 FG%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Dunks made,L5 Dunks attempted,L5 Dunk FG%,L5 Layups made,L5 Layups attempted,L5 Layup FG%,L5 Floaters made,L5 Floaters attempted,L5 Floater FG%,L5 Hook Shots made,L5 Hook Shots attempted,L5 Hook Shot FG%,L5 Jumpshots made,L5 Jumpshots attempted,L5 Jumpshot FG%,L5 Pullups made,L5 Pullups attempted,L5 Pullup FG%,L5 Turnarounds made,L5 Turnarounds attempted,L5 Turnaround FG%,L5 Step Backs made,L5 Step Backs attempted,L5 Step Back FG%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L5 1 made,L5 2-1 made,L5 2-2 made,L5 2-3 made,L5 3-1 made,L5 3-2 made,L5 3-3 made,L5 4-1 made,L5 4-2 made,L5 4-3 made,L5 1 attempted,L5 2-1 attempted,L5 2-2 attempted,L5 2-3 attempted,L5 3-1 attempted,L5 3-2 attempted,L5 3-3 attempted,L5 4-1 attempted,L5 4-2 attempted,L5 4-3 attempted,L5 zone 1 FG%,L5 zone 2-1 FG%,L5 zone 2-2 FG%,L5 zone 2-3 FG%,L5 zone 3-1 FG%,L5 zone 3-2 FG%,L5 zone 3-3 FG%,L5 zone 4-1 FG%,L5 zone 4-2 FG%,L5 zone 4-3 FG%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Dunks made,L10 Dunks attempted,L10 Dunk FG%,L10 Layups made,L10 Layups attempted,L10 Layup FG%,L10 Floaters made,L10 Floaters attempted,L10 Floater FG%,L10 Hook Shots made,L10 Hook Shots attempted,L10 Hook Shot FG%,L10 Jumpshots made,L10 Jumpshots attempted,L10 Jumpshot FG%,L10 Pullups made,L10 Pullups attempted,L10 Pullup FG%,L10 Turnarounds made,L10 Turnarounds attempted,L10 Turnaround FG%,L10 Step Backs made,L10 Step Backs attempted,L10 Step Back FG%,L10 Jumpshots assisted on,L10 Jumpshots assisted%,L10 1 made,L10 2-1 made,L10 2-2 made,L10 2-3 made,L10 3-1 made,L10 3-2 made,L10 3-3 made,L10 4-1 made,L10 4-2 made,L10 4-3 made,L10 1 attempted,L10 2-1 attempted,L10 2-2 attempted,L10 2-3 attempted,L10 3-1 attempted,L10 3

In [68]:
NAs2 = reduced_2P_df.isna().sum()/reduced_2P_df.shape[0]

In [69]:
NAs2[42::60]

L1 2PM             0.165454
L3 2PM             0.197416
L5 2PM             0.248361
L10 2PM            0.384158
TOT 2PM            0.135300
Last_season 2PM    0.333877
dtype: float64

In [70]:
reduced_2P_df.insert(41, 'Last_season Zone perc%', np.nan)
reduced_2P_df.insert(41, 'Last_season Zone attempted', 0)
reduced_2P_df.insert(41, 'Last_season Zone made', 0)
reduced_2P_df.insert(41, 'TOT Zone perc%', np.nan)
reduced_2P_df.insert(41, 'TOT Zone attempted', 0)
reduced_2P_df.insert(41, 'TOT Zone made', 0)
reduced_2P_df.insert(41, 'L10 Zone perc%', np.nan)
reduced_2P_df.insert(41, 'L10 Zone attempted', 0)
reduced_2P_df.insert(41, 'L10 Zone made', 0)
reduced_2P_df.insert(41, 'L5 Zone perc%', np.nan)
reduced_2P_df.insert(41, 'L5 Zone attempted', 0)
reduced_2P_df.insert(41, 'L5 Zone made', 0)
reduced_2P_df.insert(41, 'L3 Zone perc%', np.nan)
reduced_2P_df.insert(41, 'L3 Zone attempted', 0)
reduced_2P_df.insert(41, 'L3 Zone made', 0)
reduced_2P_df.insert(41, 'L1 Zone perc%', None)
reduced_2P_df.insert(41, 'L1 Zone attempted', 0)
reduced_2P_df.insert(41, 'L1 Zone made', 0)
reduced_2P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Dunks made,L1 Dunks attempted,L1 Dunk FG%,L1 Layups made,L1 Layups attempted,L1 Layup FG%,L1 Floaters made,L1 Floaters attempted,L1 Floater FG%,L1 Hook Shots made,L1 Hook Shots attempted,L1 Hook Shot FG%,L1 Jumpshots made,L1 Jumpshots attempted,L1 Jumpshot FG%,L1 Pullups made,L1 Pullups attempted,L1 Pullup FG%,L1 Turnarounds made,L1 Turnarounds attempted,L1 Turnaround FG%,L1 Step Backs made,L1 Step Backs attempted,L1 Step Back FG%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L1 1 made,L1 2-1 made,L1 2-2 made,L1 2-3 made,L1 3-1 made,L1 3-2 made,L1 3-3 made,L1 4-1 made,L1 4-2 made,L1 4-3 made,L1 1 attempted,L1 2-1 attempted,L1 2-2 attempted,L1 2-3 attempted,L1 3-1 attempted,L1 3-2 attempted,L1 3-3 attempted,L1 4-1 attempted,L1 4-2 attempted,L1 4-3 attempted,L1 zone 1 FG%,L1 zone 2-1 FG%,L1 zone 2-2 FG%,L1 zone 2-3 FG%,L1 zone 3-1 FG%,L1 zone 3-2 FG%,L1 zone 3-3 FG%,L1 zone 4-1 FG%,L1 zone 4-2 FG%,L1 zone 4-3 FG%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Dunks made,L3 Dunks attempted,L3 Dunk FG%,L3 Layups made,L3 Layups attempted,L3 Layup FG%,L3 Floaters made,L3 Floaters attempted,L3 Floater FG%,L3 Hook Shots made,L3 Hook Shots attempted,L3 Hook Shot FG%,L3 Jumpshots made,L3 Jumpshots attempted,L3 Jumpshot FG%,L3 Pullups made,L3 Pullups attempted,L3 Pullup FG%,L3 Turnarounds made,L3 Turnarounds attempted,L3 Turnaround FG%,L3 Step Backs made,L3 Step Backs attempted,L3 Step Back FG%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L3 1 made,L3 2-1 made,L3 2-2 made,L3 2-3 made,L3 3-1 made,L3 3-2 made,L3 3-3 made,L3 4-1 made,L3 4-2 made,L3 4-3 made,L3 1 attempted,L3 2-1 attempted,L3 2-2 attempted,L3 2-3 attempted,L3 3-1 attempted,L3 3-2 attempted,L3 3-3 attempted,L3 4-1 attempted,L3 4-2 attempted,L3 4-3 attempted,L3 zone 1 FG%,L3 zone 2-1 FG%,L3 zone 2-2 FG%,L3 zone 2-3 FG%,L3 zone 3-1 FG%,L3 zone 3-2 FG%,L3 zone 3-3 FG%,L3 zone 4-1 FG%,L3 zone 4-2 FG%,L3 zone 4-3 FG%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Dunks made,L5 Dunks attempted,L5 Dunk FG%,L5 Layups made,L5 Layups attempted,L5 Layup FG%,L5 Floaters made,L5 Floaters attempted,L5 Floater FG%,L5 Hook Shots made,L5 Hook Shots attempted,L5 Hook Shot FG%,L5 Jumpshots made,L5 Jumpshots attempted,L5 Jumpshot FG%,L5 Pullups made,L5 Pullups attempted,L5 Pullup FG%,L5 Turnarounds made,L5 Turnarounds attempted,L5 Turnaround FG%,L5 Step Backs made,L5 Step Backs attempted,L5 Step Back FG%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L5 1 made,L5 2-1 made,L5 2-2 made,L5 2-3 made,L5 3-1 made,L5 3-2 made,L5 3-3 made,L5 4-1 made,L5 4-2 made,L5 4-3 made,L5 1 attempted,L5 2-1 attempted,L5 2-2 attempted,L5 2-3 attempted,L5 3-1 attempted,L5 3-2 attempted,L5 3-3 attempted,L5 4-1 attempted,L5 4-2 attempted,L5 4-3 attempted,L5 zone 1 FG%,L5 zone 2-1 FG%,L5 zone 2-2 FG%,L5 zone 2-3 FG%,L5 zone 3-1 FG%,L5 zone 3-2 FG%,L5 zone 3-3 FG%,L5 zone 4-1 FG%,L5 zone 4-2 FG%,L5 zone 4-3 FG%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Dunks made,L10 Dunks attempted,L10 Dunk FG%,L10 Layups made,L10 Layups attempted,L10 Layup FG%,L10 Floaters made,L10 Floaters attempted,L10 Floater FG%,L10 Hook Shots made,L10 Hook Shots attempted,L10 Hook Shot FG%,L10 Jumpshots made,L10 Jumpshots attempted,L10 Jumpshot FG%,L10 Pullups made,L10 Pullups attempted,L10 Pullup FG%,L10 Turnarounds made,L10 Turnarounds attempted,L10 Turnaround FG%,L10 Step Backs made,L10 Step Ba

In [71]:
cols_to_drop = list()

for time in ['L1','L3','L5','L10','TOT','Last_season']:
    for zone in [' 1',' 2-1',' 2-2',' 2-3',' 3-1',' 3-2',' 3-3',' 4-1',' 4-2',' 4-3']:
        reduced_2P_df[f"{time} Zone made"] = np.where(reduced_2P_df.Zone==zone, reduced_2P_df[f"{time}{zone} made"], reduced_2P_df[f"{time} Zone made"])
        cols_to_drop.append(f"{time}{zone} made")
        reduced_2P_df[f"{time} Zone attempted"] = np.where(reduced_2P_df.Zone==zone, reduced_2P_df[f"{time}{zone} attempted"], reduced_2P_df[f"{time} Zone attempted"])
        cols_to_drop.append(f"{time}{zone} attempted")
        reduced_2P_df[f"{time} Zone perc%"] = np.where(reduced_2P_df.Zone==zone, reduced_2P_df[f"{time} zone{zone} FG%"], reduced_2P_df[f"{time} Zone perc%"])
        cols_to_drop.append(f"{time} zone{zone} FG%")

reduced_2P_df
            

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Dunks made,L1 Dunks attempted,L1 Dunk FG%,L1 Layups made,L1 Layups attempted,L1 Layup FG%,L1 Floaters made,L1 Floaters attempted,L1 Floater FG%,L1 Hook Shots made,L1 Hook Shots attempted,L1 Hook Shot FG%,L1 Jumpshots made,L1 Jumpshots attempted,L1 Jumpshot FG%,L1 Pullups made,L1 Pullups attempted,L1 Pullup FG%,L1 Turnarounds made,L1 Turnarounds attempted,L1 Turnaround FG%,L1 Step Backs made,L1 Step Backs attempted,L1 Step Back FG%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L1 1 made,L1 2-1 made,L1 2-2 made,L1 2-3 made,L1 3-1 made,L1 3-2 made,L1 3-3 made,L1 4-1 made,L1 4-2 made,L1 4-3 made,L1 1 attempted,L1 2-1 attempted,L1 2-2 attempted,L1 2-3 attempted,L1 3-1 attempted,L1 3-2 attempted,L1 3-3 attempted,L1 4-1 attempted,L1 4-2 attempted,L1 4-3 attempted,L1 zone 1 FG%,L1 zone 2-1 FG%,L1 zone 2-2 FG%,L1 zone 2-3 FG%,L1 zone 3-1 FG%,L1 zone 3-2 FG%,L1 zone 3-3 FG%,L1 zone 4-1 FG%,L1 zone 4-2 FG%,L1 zone 4-3 FG%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Dunks made,L3 Dunks attempted,L3 Dunk FG%,L3 Layups made,L3 Layups attempted,L3 Layup FG%,L3 Floaters made,L3 Floaters attempted,L3 Floater FG%,L3 Hook Shots made,L3 Hook Shots attempted,L3 Hook Shot FG%,L3 Jumpshots made,L3 Jumpshots attempted,L3 Jumpshot FG%,L3 Pullups made,L3 Pullups attempted,L3 Pullup FG%,L3 Turnarounds made,L3 Turnarounds attempted,L3 Turnaround FG%,L3 Step Backs made,L3 Step Backs attempted,L3 Step Back FG%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L3 1 made,L3 2-1 made,L3 2-2 made,L3 2-3 made,L3 3-1 made,L3 3-2 made,L3 3-3 made,L3 4-1 made,L3 4-2 made,L3 4-3 made,L3 1 attempted,L3 2-1 attempted,L3 2-2 attempted,L3 2-3 attempted,L3 3-1 attempted,L3 3-2 attempted,L3 3-3 attempted,L3 4-1 attempted,L3 4-2 attempted,L3 4-3 attempted,L3 zone 1 FG%,L3 zone 2-1 FG%,L3 zone 2-2 FG%,L3 zone 2-3 FG%,L3 zone 3-1 FG%,L3 zone 3-2 FG%,L3 zone 3-3 FG%,L3 zone 4-1 FG%,L3 zone 4-2 FG%,L3 zone 4-3 FG%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Dunks made,L5 Dunks attempted,L5 Dunk FG%,L5 Layups made,L5 Layups attempted,L5 Layup FG%,L5 Floaters made,L5 Floaters attempted,L5 Floater FG%,L5 Hook Shots made,L5 Hook Shots attempted,L5 Hook Shot FG%,L5 Jumpshots made,L5 Jumpshots attempted,L5 Jumpshot FG%,L5 Pullups made,L5 Pullups attempted,L5 Pullup FG%,L5 Turnarounds made,L5 Turnarounds attempted,L5 Turnaround FG%,L5 Step Backs made,L5 Step Backs attempted,L5 Step Back FG%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L5 1 made,L5 2-1 made,L5 2-2 made,L5 2-3 made,L5 3-1 made,L5 3-2 made,L5 3-3 made,L5 4-1 made,L5 4-2 made,L5 4-3 made,L5 1 attempted,L5 2-1 attempted,L5 2-2 attempted,L5 2-3 attempted,L5 3-1 attempted,L5 3-2 attempted,L5 3-3 attempted,L5 4-1 attempted,L5 4-2 attempted,L5 4-3 attempted,L5 zone 1 FG%,L5 zone 2-1 FG%,L5 zone 2-2 FG%,L5 zone 2-3 FG%,L5 zone 3-1 FG%,L5 zone 3-2 FG%,L5 zone 3-3 FG%,L5 zone 4-1 FG%,L5 zone 4-2 FG%,L5 zone 4-3 FG%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Dunks made,L10 Dunks attempted,L10 Dunk FG%,L10 Layups made,L10 Layups attempted,L10 Layup FG%,L10 Floaters made,L10 Floaters attempted,L10 Floater FG%,L10 Hook Shots made,L10 Hook Shots attempted,L10 Hook Shot FG%,L10 Jumpshots made,L10 Jumpshots attempted,L10 Jumpshot FG%,L10 Pullups made,L10 Pullups attempted,L10 Pullup FG%,L10 Turnarounds made,L10 Turnarounds attempted,L10 Turnaround FG%,L10 Step Backs made,L10 Step Ba

In [72]:
reduced_2P_df = reduced_2P_df.drop(columns=cols_to_drop)
reduced_2P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Dunks made,L1 Dunks attempted,L1 Dunk FG%,L1 Layups made,L1 Layups attempted,L1 Layup FG%,L1 Floaters made,L1 Floaters attempted,L1 Floater FG%,L1 Hook Shots made,L1 Hook Shots attempted,L1 Hook Shot FG%,L1 Jumpshots made,L1 Jumpshots attempted,L1 Jumpshot FG%,L1 Pullups made,L1 Pullups attempted,L1 Pullup FG%,L1 Turnarounds made,L1 Turnarounds attempted,L1 Turnaround FG%,L1 Step Backs made,L1 Step Backs attempted,L1 Step Back FG%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Dunks made,L3 Dunks attempted,L3 Dunk FG%,L3 Layups made,L3 Layups attempted,L3 Layup FG%,L3 Floaters made,L3 Floaters attempted,L3 Floater FG%,L3 Hook Shots made,L3 Hook Shots attempted,L3 Hook Shot FG%,L3 Jumpshots made,L3 Jumpshots attempted,L3 Jumpshot FG%,L3 Pullups made,L3 Pullups attempted,L3 Pullup FG%,L3 Turnarounds made,L3 Turnarounds attempted,L3 Turnaround FG%,L3 Step Backs made,L3 Step Backs attempted,L3 Step Back FG%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Dunks made,L5 Dunks attempted,L5 Dunk FG%,L5 Layups made,L5 Layups attempted,L5 Layup FG%,L5 Floaters made,L5 Floaters attempted,L5 Floater FG%,L5 Hook Shots made,L5 Hook Shots attempted,L5 Hook Shot FG%,L5 Jumpshots made,L5 Jumpshots attempted,L5 Jumpshot FG%,L5 Pullups made,L5 Pullups attempted,L5 Pullup FG%,L5 Turnarounds made,L5 Turnarounds attempted,L5 Turnaround FG%,L5 Step Backs made,L5 Step Backs attempted,L5 Step Back FG%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Dunks made,L10 Dunks attempted,L10 Dunk FG%,L10 Layups made,L10 Layups attempted,L10 Layup FG%,L10 Floaters made,L10 Floaters attempted,L10 Floater FG%,L10 Hook Shots made,L10 Hook Shots attempted,L10 Hook Shot FG%,L10 Jumpshots made,L10 Jumpshots attempted,L10 Jumpshot FG%,L10 Pullups made,L10 Pullups attempted,L10 Pullup FG%,L10 Turnarounds made,L10 Turnarounds attempted,L10 Turnaround FG%,L10 Step Backs made,L10 Step Backs attempted,L10 Step Back FG%,L10 Jumpshots assisted on,L10 Jumpshots assisted%,TOT played,TOT 2PM,TOT 2PA,TOT 2P%,TOT Dunks made,TOT Dunks attempted,TOT Dunk FG%,TOT Layups made,TOT Layups attempted,TOT Layup FG%,TOT Floaters made,TOT Floaters attempted,TOT Floater FG%,TOT Hook Shots made,TOT Hook Shots attempted,TOT Hook Shot FG%,TOT Jumpshots made,TOT Jumpshots attempted,TOT Jumpshot FG%,TOT Pullups made,TOT Pullups attempted,TOT Pullup FG%,TOT Turnarounds made,TOT Turnarounds attempted,TOT Turnaround FG%,TOT Step Backs made,TOT Step Backs attempted,TOT Step Back FG%,TOT Jumpshots assisted on,TOT Jumpshots assisted%,Last_season played,Last_season 2PM,Last_season 2PA,Last_season 2P%,Last_season Dunks made,Last_season Dunks attempted,Last_season Dunk FG%,Last_season Layups made,Last_season Layups attempted,Last_season Layup FG%,Last_season Floaters made,Last_season Floaters attempted,Last_season Floater FG%,Last_season Hook Shots made,Last_season Hook Shots attempted,Last_season Hook Shot FG%,Last_season Jumpshots made,Last_season Jumpshots attempted,Last_season Jumpshot FG%,Last_season Pullups made,Last_season Pullups attempted,Last_season Pullup FG%,Last_season Turnarounds made,Last_season Turnarounds attempted,Last_season Turnaround FG%,Last_season Step Backs made,Last_season Step Backs attempted,Last_sea

In [73]:
reduced_2P_df.insert(56, 'Shot type', 'Other')
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Jumpshot==True, 'Jumpshot', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Pullup==True, 'Pullup', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df['Step Back']==True, 'Step Back', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Floater==True, 'Floater', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Layup==True, 'Layup', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df['Hook Shot']==True, 'Hook Shot', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Dunk==True, 'Dunk', reduced_2P_df['Shot type'])
reduced_2P_df['Shot type'] = np.where(reduced_2P_df.Turnaround==True, 'Turnaround', reduced_2P_df['Shot type'])
reduced_2P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Shot type,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Dunks made,L1 Dunks attempted,L1 Dunk FG%,L1 Layups made,L1 Layups attempted,L1 Layup FG%,L1 Floaters made,L1 Floaters attempted,L1 Floater FG%,L1 Hook Shots made,L1 Hook Shots attempted,L1 Hook Shot FG%,L1 Jumpshots made,L1 Jumpshots attempted,L1 Jumpshot FG%,L1 Pullups made,L1 Pullups attempted,L1 Pullup FG%,L1 Turnarounds made,L1 Turnarounds attempted,L1 Turnaround FG%,L1 Step Backs made,L1 Step Backs attempted,L1 Step Back FG%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Dunks made,L3 Dunks attempted,L3 Dunk FG%,L3 Layups made,L3 Layups attempted,L3 Layup FG%,L3 Floaters made,L3 Floaters attempted,L3 Floater FG%,L3 Hook Shots made,L3 Hook Shots attempted,L3 Hook Shot FG%,L3 Jumpshots made,L3 Jumpshots attempted,L3 Jumpshot FG%,L3 Pullups made,L3 Pullups attempted,L3 Pullup FG%,L3 Turnarounds made,L3 Turnarounds attempted,L3 Turnaround FG%,L3 Step Backs made,L3 Step Backs attempted,L3 Step Back FG%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Dunks made,L5 Dunks attempted,L5 Dunk FG%,L5 Layups made,L5 Layups attempted,L5 Layup FG%,L5 Floaters made,L5 Floaters attempted,L5 Floater FG%,L5 Hook Shots made,L5 Hook Shots attempted,L5 Hook Shot FG%,L5 Jumpshots made,L5 Jumpshots attempted,L5 Jumpshot FG%,L5 Pullups made,L5 Pullups attempted,L5 Pullup FG%,L5 Turnarounds made,L5 Turnarounds attempted,L5 Turnaround FG%,L5 Step Backs made,L5 Step Backs attempted,L5 Step Back FG%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Dunks made,L10 Dunks attempted,L10 Dunk FG%,L10 Layups made,L10 Layups attempted,L10 Layup FG%,L10 Floaters made,L10 Floaters attempted,L10 Floater FG%,L10 Hook Shots made,L10 Hook Shots attempted,L10 Hook Shot FG%,L10 Jumpshots made,L10 Jumpshots attempted,L10 Jumpshot FG%,L10 Pullups made,L10 Pullups attempted,L10 Pullup FG%,L10 Turnarounds made,L10 Turnarounds attempted,L10 Turnaround FG%,L10 Step Backs made,L10 Step Backs attempted,L10 Step Back FG%,L10 Jumpshots assisted on,L10 Jumpshots assisted%,TOT played,TOT 2PM,TOT 2PA,TOT 2P%,TOT Dunks made,TOT Dunks attempted,TOT Dunk FG%,TOT Layups made,TOT Layups attempted,TOT Layup FG%,TOT Floaters made,TOT Floaters attempted,TOT Floater FG%,TOT Hook Shots made,TOT Hook Shots attempted,TOT Hook Shot FG%,TOT Jumpshots made,TOT Jumpshots attempted,TOT Jumpshot FG%,TOT Pullups made,TOT Pullups attempted,TOT Pullup FG%,TOT Turnarounds made,TOT Turnarounds attempted,TOT Turnaround FG%,TOT Step Backs made,TOT Step Backs attempted,TOT Step Back FG%,TOT Jumpshots assisted on,TOT Jumpshots assisted%,Last_season played,Last_season 2PM,Last_season 2PA,Last_season 2P%,Last_season Dunks made,Last_season Dunks attempted,Last_season Dunk FG%,Last_season Layups made,Last_season Layups attempted,Last_season Layup FG%,Last_season Floaters made,Last_season Floaters attempted,Last_season Floater FG%,Last_season Hook Shots made,Last_season Hook Shots attempted,Last_season Hook Shot FG%,Last_season Jumpshots made,Last_season Jumpshots attempted,Last_season Jumpshot FG%,Last_season Pullups made,Last_season Pullups attempted,Last_season Pullup FG%,Last_season Turnarounds made,Last_season Turnarounds attempted,Last_season Turnaround FG%,Last_season Step Backs made,Last_season Step Backs attempte

In [74]:
reduced_2P_df['Shot type'].value_counts()

Layup         233708
Jumpshot      103745
Dunk           23426
Pullup         21723
Floater        16052
Hook Shot      14865
Turnaround     13943
Step Back       6131
Other           2735
Name: Shot type, dtype: int64

In [75]:
reduced_2P_df.insert(57, 'Last_season Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'Last_season Shot type attempted', 0)
reduced_2P_df.insert(57, 'Last_season Shot type made', 0)
reduced_2P_df.insert(57, 'TOT Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'TOT Shot type attempted', 0)
reduced_2P_df.insert(57, 'TOT Shot type made', 0)
reduced_2P_df.insert(57, 'L10 Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'L10 Shot type attempted', 0)
reduced_2P_df.insert(57, 'L10 Shot type made', 0)
reduced_2P_df.insert(57, 'L5 Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'L5 Shot type attempted', 0)
reduced_2P_df.insert(57, 'L5 Shot type made', 0)
reduced_2P_df.insert(57, 'L3 Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'L3 Shot type attempted', 0)
reduced_2P_df.insert(57, 'L3 Shot type made', 0)
reduced_2P_df.insert(57, 'L1 Shot type perc%', np.nan)
reduced_2P_df.insert(57, 'L1 Shot type attempted', 0)
reduced_2P_df.insert(57, 'L1 Shot type made', 0)
reduced_2P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Shot type,L1 Shot type made,L1 Shot type attempted,L1 Shot type perc%,L3 Shot type made,L3 Shot type attempted,L3 Shot type perc%,L5 Shot type made,L5 Shot type attempted,L5 Shot type perc%,L10 Shot type made,L10 Shot type attempted,L10 Shot type perc%,TOT Shot type made,TOT Shot type attempted,TOT Shot type perc%,Last_season Shot type made,Last_season Shot type attempted,Last_season Shot type perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Dunks made,L1 Dunks attempted,L1 Dunk FG%,L1 Layups made,L1 Layups attempted,L1 Layup FG%,L1 Floaters made,L1 Floaters attempted,L1 Floater FG%,L1 Hook Shots made,L1 Hook Shots attempted,L1 Hook Shot FG%,L1 Jumpshots made,L1 Jumpshots attempted,L1 Jumpshot FG%,L1 Pullups made,L1 Pullups attempted,L1 Pullup FG%,L1 Turnarounds made,L1 Turnarounds attempted,L1 Turnaround FG%,L1 Step Backs made,L1 Step Backs attempted,L1 Step Back FG%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Dunks made,L3 Dunks attempted,L3 Dunk FG%,L3 Layups made,L3 Layups attempted,L3 Layup FG%,L3 Floaters made,L3 Floaters attempted,L3 Floater FG%,L3 Hook Shots made,L3 Hook Shots attempted,L3 Hook Shot FG%,L3 Jumpshots made,L3 Jumpshots attempted,L3 Jumpshot FG%,L3 Pullups made,L3 Pullups attempted,L3 Pullup FG%,L3 Turnarounds made,L3 Turnarounds attempted,L3 Turnaround FG%,L3 Step Backs made,L3 Step Backs attempted,L3 Step Back FG%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Dunks made,L5 Dunks attempted,L5 Dunk FG%,L5 Layups made,L5 Layups attempted,L5 Layup FG%,L5 Floaters made,L5 Floaters attempted,L5 Floater FG%,L5 Hook Shots made,L5 Hook Shots attempted,L5 Hook Shot FG%,L5 Jumpshots made,L5 Jumpshots attempted,L5 Jumpshot FG%,L5 Pullups made,L5 Pullups attempted,L5 Pullup FG%,L5 Turnarounds made,L5 Turnarounds attempted,L5 Turnaround FG%,L5 Step Backs made,L5 Step Backs attempted,L5 Step Back FG%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Dunks made,L10 Dunks attempted,L10 Dunk FG%,L10 Layups made,L10 Layups attempted,L10 Layup FG%,L10 Floaters made,L10 Floaters attempted,L10 Floater FG%,L10 Hook Shots made,L10 Hook Shots attempted,L10 Hook Shot FG%,L10 Jumpshots made,L10 Jumpshots attempted,L10 Jumpshot FG%,L10 Pullups made,L10 Pullups attempted,L10 Pullup FG%,L10 Turnarounds made,L10 Turnarounds attempted,L10 Turnaround FG%,L10 Step Backs made,L10 Step Backs attempted,L10 Step Back FG%,L10 Jumpshots assisted on,L10 Jumpshots assisted%,TOT played,TOT 2PM,TOT 2PA,TOT 2P%,TOT Dunks made,TOT Dunks attempted,TOT Dunk FG%,TOT Layups made,TOT Layups attempted,TOT Layup FG%,TOT Floaters made,TOT Floaters attempted,TOT Floater FG%,TOT Hook Shots made,TOT Hook Shots attempted,TOT Hook Shot FG%,TOT Jumpshots made,TOT Jumpshots attempted,TOT Jumpshot FG%,TOT Pullups made,TOT Pullups attempted,TOT Pullup FG%,TOT Turnarounds made,TOT Turnarounds attempted,TOT Turnaround FG%,TOT Step Backs made,TOT Step Backs attempted,TOT Step Back FG%,TOT Jumpshots assisted on,TOT Jumpshots assisted%,Last_season played,Last_season 2PM,Last_season 2PA,Last_season 2P%,Last_season Dunks made,Last_season Dunks attempted,Last_season Dunk FG%,Last_season Layups made,Last_season Layups attempted,Last_season Layup FG%,Last_season Floaters made,Last_season Floaters attempted,Last_season Floater FG%,Last_

In [76]:
cols_to_drop = list()

for time in ['L1','L3','L5','L10','TOT','Last_season']:
    for shot_type in ['Dunk','Hook Shot','Layup','Floater','Jumpshot','Pullup','Turnaround','Step Back']:
        reduced_2P_df[f"{time} Shot type made"] = np.where(reduced_2P_df['Shot type']==shot_type, reduced_2P_df[f"{time} {shot_type}s made"], reduced_2P_df[f"{time} Shot type made"])
        cols_to_drop.append(f"{time} {shot_type}s made")
        reduced_2P_df[f"{time} Shot type attempted"] = np.where(reduced_2P_df['Shot type']==shot_type, reduced_2P_df[f"{time} {shot_type}s attempted"], reduced_2P_df[f"{time} Shot type attempted"])
        cols_to_drop.append(f"{time} {shot_type}s attempted")
        reduced_2P_df[f"{time} Shot type perc%"] = np.where(reduced_2P_df['Shot type']==shot_type, reduced_2P_df[f"{time} {shot_type} FG%"], reduced_2P_df[f"{time} Shot type perc%"])
        cols_to_drop.append(f"{time} {shot_type} FG%")

reduced_2P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Shot type,L1 Shot type made,L1 Shot type attempted,L1 Shot type perc%,L3 Shot type made,L3 Shot type attempted,L3 Shot type perc%,L5 Shot type made,L5 Shot type attempted,L5 Shot type perc%,L10 Shot type made,L10 Shot type attempted,L10 Shot type perc%,TOT Shot type made,TOT Shot type attempted,TOT Shot type perc%,Last_season Shot type made,Last_season Shot type attempted,Last_season Shot type perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Dunks made,L1 Dunks attempted,L1 Dunk FG%,L1 Layups made,L1 Layups attempted,L1 Layup FG%,L1 Floaters made,L1 Floaters attempted,L1 Floater FG%,L1 Hook Shots made,L1 Hook Shots attempted,L1 Hook Shot FG%,L1 Jumpshots made,L1 Jumpshots attempted,L1 Jumpshot FG%,L1 Pullups made,L1 Pullups attempted,L1 Pullup FG%,L1 Turnarounds made,L1 Turnarounds attempted,L1 Turnaround FG%,L1 Step Backs made,L1 Step Backs attempted,L1 Step Back FG%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Dunks made,L3 Dunks attempted,L3 Dunk FG%,L3 Layups made,L3 Layups attempted,L3 Layup FG%,L3 Floaters made,L3 Floaters attempted,L3 Floater FG%,L3 Hook Shots made,L3 Hook Shots attempted,L3 Hook Shot FG%,L3 Jumpshots made,L3 Jumpshots attempted,L3 Jumpshot FG%,L3 Pullups made,L3 Pullups attempted,L3 Pullup FG%,L3 Turnarounds made,L3 Turnarounds attempted,L3 Turnaround FG%,L3 Step Backs made,L3 Step Backs attempted,L3 Step Back FG%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Dunks made,L5 Dunks attempted,L5 Dunk FG%,L5 Layups made,L5 Layups attempted,L5 Layup FG%,L5 Floaters made,L5 Floaters attempted,L5 Floater FG%,L5 Hook Shots made,L5 Hook Shots attempted,L5 Hook Shot FG%,L5 Jumpshots made,L5 Jumpshots attempted,L5 Jumpshot FG%,L5 Pullups made,L5 Pullups attempted,L5 Pullup FG%,L5 Turnarounds made,L5 Turnarounds attempted,L5 Turnaround FG%,L5 Step Backs made,L5 Step Backs attempted,L5 Step Back FG%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Dunks made,L10 Dunks attempted,L10 Dunk FG%,L10 Layups made,L10 Layups attempted,L10 Layup FG%,L10 Floaters made,L10 Floaters attempted,L10 Floater FG%,L10 Hook Shots made,L10 Hook Shots attempted,L10 Hook Shot FG%,L10 Jumpshots made,L10 Jumpshots attempted,L10 Jumpshot FG%,L10 Pullups made,L10 Pullups attempted,L10 Pullup FG%,L10 Turnarounds made,L10 Turnarounds attempted,L10 Turnaround FG%,L10 Step Backs made,L10 Step Backs attempted,L10 Step Back FG%,L10 Jumpshots assisted on,L10 Jumpshots assisted%,TOT played,TOT 2PM,TOT 2PA,TOT 2P%,TOT Dunks made,TOT Dunks attempted,TOT Dunk FG%,TOT Layups made,TOT Layups attempted,TOT Layup FG%,TOT Floaters made,TOT Floaters attempted,TOT Floater FG%,TOT Hook Shots made,TOT Hook Shots attempted,TOT Hook Shot FG%,TOT Jumpshots made,TOT Jumpshots attempted,TOT Jumpshot FG%,TOT Pullups made,TOT Pullups attempted,TOT Pullup FG%,TOT Turnarounds made,TOT Turnarounds attempted,TOT Turnaround FG%,TOT Step Backs made,TOT Step Backs attempted,TOT Step Back FG%,TOT Jumpshots assisted on,TOT Jumpshots assisted%,Last_season played,Last_season 2PM,Last_season 2PA,Last_season 2P%,Last_season Dunks made,Last_season Dunks attempted,Last_season Dunk FG%,Last_season Layups made,Last_season Layups attempted,Last_season Layup FG%,Last_season Floaters made,Last_season Floaters attempted,Last_season Floater FG%,Last_

In [77]:
reduced_2P_df = reduced_2P_df.drop(columns=cols_to_drop)
reduced_2P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Shot type,L1 Shot type made,L1 Shot type attempted,L1 Shot type perc%,L3 Shot type made,L3 Shot type attempted,L3 Shot type perc%,L5 Shot type made,L5 Shot type attempted,L5 Shot type perc%,L10 Shot type made,L10 Shot type attempted,L10 Shot type perc%,TOT Shot type made,TOT Shot type attempted,TOT Shot type perc%,Last_season Shot type made,Last_season Shot type attempted,Last_season Shot type perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Jumpshots assisted on,L10 Jumpshots assisted%,TOT played,TOT 2PM,TOT 2PA,TOT 2P%,TOT Jumpshots assisted on,TOT Jumpshots assisted%,Last_season played,Last_season 2PM,Last_season 2PA,Last_season 2P%,Last_season Jumpshots assisted on,Last_season Jumpshots assisted%
0,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,636a5d88531336e79c342589,636a5fa6531336e79c3425e1,3,2,1,636a5f91531336e79c3425b4,18:11,1091,TwoMiss,4-2,Two,0,21.0,20.7,75.5,15.94,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Floater,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,636a5d88531336e79c34258a,636a5fa6531336e79c3425e3,3,2,1,636a5f91531336e79c3425b4,18:6,1086,TwoMiss,3-1,Two,0,14.0,11.3,28.7,12.54,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f45ed05764db467fe4e,636a5fa6531336e79c3425e8,3,4,1,636a5f91531336e79c3425b5,17:7,1027,TwoMiss,3-2,Two,0,25.0,16.9,90.0,11.67,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Layup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.384615,0.923077,0.416667,0.000000,0.076923,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,0.576923,1.576923,0.365854,2.0,0.133333
3,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,636a5d88531336e79c342588,636a5fa6531336e79c3425ea,3,4,1,636a5f91531336e79c3425b5,16:28,988,TwoMiss,4-2,Two,0,25.0,24.4,90.0,19.19,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [81]:
condensed_per_game_2P_features = list(range(-73,0))
condensed_per_game_2P_features.insert(0,38)
condensed_per_game_2P_features.insert(0,4)

In [82]:
reduced_2P_df.iloc[:,condensed_per_game_2P_features].to_pickle("Condensed_2P_features_df.pickle")
reduced_2P_df.iloc[:,condensed_per_game_2P_features]

,shot_id,D1,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Shot type,L1 Shot type made,L1 Shot type attempted,L1 Shot type perc%,L3 Shot type made,L3 Shot type attempted,L3 Shot type perc%,L5 Shot type made,L5 Shot type attempted,L5 Shot type perc%,L10 Shot type made,L10 Shot type attempted,L10 Shot type perc%,TOT Shot type made,TOT Shot type attempted,TOT Shot type perc%,Last_season Shot type made,Last_season Shot type attempted,Last_season Shot type perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 2PM,L1 2PA,L1 2P%,L1 Jumpshots assisted on,L1 Jumpshots assisted%,L3 played,L3 2PM,L3 2PA,L3 2P%,L3 Jumpshots assisted on,L3 Jumpshots assisted%,L5 played,L5 2PM,L5 2PA,L5 2P%,L5 Jumpshots assisted on,L5 Jumpshots assisted%,L10 played,L10 2PM,L10 2PA,L10 2P%,L10 Jumpshots assisted on,L10 Jumpshots assisted%,TOT played,TOT 2PM,TOT 2PA,TOT 2P%,TOT Jumpshots assisted on,TOT Jumpshots assisted%,Last_season played,Last_season 2PM,Last_season 2PA,Last_season 2P%,Last_season Jumpshots assisted on,Last_season Jumpshots assisted%
0,636a5fa6531336e79c3425e1,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Floater,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,636a5fa6531336e79c3425e3,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,636a5fa6531336e79c3425e8,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Layup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.384615,0.923077,0.416667,0.000000,0.076923,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,0.576923,1.576923,0.365854,2.0,0.133333
3,636a5fa6531336e79c3425ea,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,636a5fa6531336e79c3425ee,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Layup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.411765,2.764706,0.510638,0.176471,0.294118,0.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,1.705882,3.294118,0.517857,1.0,0.034483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436323,648b3a7b98debb5769861cca,True,1.0,5.0,0.2,3.000000,5.000000,0.6,2.2,4.6,0.478261,2.0,4.2,0.476190,1.655172,3.103448,0.533333,Layup,1.0,6.0,0.166667,2.666667,6.666667,0.4,2.0,5.4,0.370370,1.9,4.7,0.404255,1.862069,3.931034,0.473684,1.516129,3.516129,0.431193,1.387097,2.870968,0.483146,1.0,2.0,8.0,0.25,0.0,0.0,3.0,3.333333,10.000000,0.333333,0.0,0.0,5.0,2.6,8.6,0.302326,1.0,0.076923,10.0,2.7,7.4,0.364865,1.0,0.037037,29.0,2.655172,6.344828,0.418478,2.0,0.025974,31.0,2.322581,5.580645,0.416185,1.0,0.013889
436324,648b3a7b98debb5769861cdd,True,2.0,3.0,0.666667,0.666667,1.333333,0.5,1.0,2.0,0.500000,1.3,2.3,0.565217,1.035714,2.214286,0.467742,Layup,2.0,3.0,0.666667,1.000000,1.666667,0.6,

Feature creation for 3 pointers

In [83]:
#Filter on 3 point shots
Threes_df = shots_df[shots_df.Type=='Three']
three_pointers_df = Threes_df.groupby(['player_id','game_id']).Result.agg(
    ['sum','count']).reset_index().rename(columns={'sum':'3PM', 'count':'3PA'})
three_pointers_df

,player_id,game_id,3PM,3PA
0,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,6,6
1,618bf7f3815098ca6660f334,636f9e5827131d1edcad44ed,0,3
2,618bf7f3815098ca6660f334,637310726c21c0a3cfa28625,3,6
3,618bf7f3815098ca6660f334,6378674f4650d0358704664d,5,7
4,618bf7f3815098ca6660f334,637c5431fdedb41779d81fcc,4,9
...,...,...,...,...
78511,640004f5effffd352c7ea68e,640004fbeffffd352c7eab2e,0,2
78512,640004f5effffd352c7ea68f,640004fbeffffd352c7eab2e,2,6
78513,640004f5effffd352c7ea690,640004fbeffffd352c7eab2e,0,1
78514,64018833f25daee41501b666,64018839f25daee41501ba68,0,1


In [84]:
#merge with the dataframe containing the team_id, game_id, player_id combinations
three_pointers_df = pd.merge(team_player_game_df, three_pointers_df, how= 'left', on=['player_id','game_id']).fillna(0)
three_pointers_df

,team_id,player_id,game_id,games played since,played,3PM,3PA
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0,0.0,0.0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1,0,0.0,0.0
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2,0,0.0,0.0
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3,0,0.0,0.0
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4,1,0.0,0.0
...,...,...,...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29,0,0.0,0.0
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30,0,0.0,0.0
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31,0,0.0,0.0
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32,0,0.0,0.0


In [85]:
#Create made and attempted game totals for each of the following shot types
for stat in ['Jumpshot', 'Pullup', 'Step Back']:
    temp = Threes_df[Threes_df[stat]==True]
    temp_df = temp.groupby(['player_id','game_id']).Result.agg(
        ['sum','count']).reset_index().rename(columns={'sum': '3P '+stat+'s made', 'count': '3P '+stat+'s attempted'})
    three_pointers_df = pd.merge(three_pointers_df, temp_df, how= 'left', on=['player_id','game_id']).fillna(0)
three_pointers_df

,team_id,player_id,game_id,games played since,played,3PM,3PA,3P Jumpshots made,3P Jumpshots attempted,3P Pullups made,3P Pullups attempted,3P Step Backs made,3P Step Backs attempted
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
#Calculated percentage of made 3p jumpshots that were assisted
temp_df = Threes_df[(Threes_df['Jumpshot']==True)&(Threes_df['Result']==1)].groupby(
    ['player_id','game_id'])['Assisted'].sum().reset_index().rename(columns={'Assisted': '3P Jumpshots assisted on'})
temp_df

,player_id,game_id,3P Jumpshots assisted on
0,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,4
1,618bf7f3815098ca6660f334,637310726c21c0a3cfa28625,1
2,618bf7f3815098ca6660f334,6378674f4650d0358704664d,4
3,618bf7f3815098ca6660f334,637c5431fdedb41779d81fcc,4
4,618bf7f3815098ca6660f334,637effe51280212ca084ca78,3
...,...,...,...
46207,640004f5effffd352c7ea689,640004fbeffffd352c7eab2e,0
46208,640004f5effffd352c7ea68a,640004fbeffffd352c7eab2e,1
46209,640004f5effffd352c7ea68b,640004fbeffffd352c7eab2e,1
46210,640004f5effffd352c7ea68d,640004fbeffffd352c7eab2e,1


In [87]:
three_pointers_df = pd.merge(three_pointers_df, temp_df, how= 'left', on=['player_id','game_id']).fillna(0)
three_pointers_df

,team_id,player_id,game_id,games played since,played,3PM,3PA,3P Jumpshots made,3P Jumpshots attempted,3P Pullups made,3P Pullups attempted,3P Step Backs made,3P Step Backs attempted,3P Jumpshots assisted on
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
#3P% per zone for each player
Zones3 = Threes_df.groupby(['player_id','game_id','Zone']).Result.agg(['sum','count']).reset_index()
Zones3

,player_id,game_id,Zone,sum,count
0,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,6-2,1,1
1,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,7-1,1,1
2,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,7-2,2,2
3,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,7-3,2,2
4,618bf7f3815098ca6660f334,636f9e5827131d1edcad44ed,6-1,0,1
...,...,...,...,...,...
185555,640004f5effffd352c7ea68f,640004fbeffffd352c7eab2e,7-2,1,4
185556,640004f5effffd352c7ea68f,640004fbeffffd352c7eab2e,7-3,0,1
185557,640004f5effffd352c7ea690,640004fbeffffd352c7eab2e,6-3,0,1
185558,64018833f25daee41501b666,64018839f25daee41501ba68,5-1,0,1


In [89]:
#Pivot on Zone column before merging with the features dataframe
pivot_zones = Zones3.pivot(index=['player_id','game_id'], columns='Zone')[['sum','count']].reset_index().rename(
    columns={'sum':'made','count':'attempted'})
pivot_zones.columns = pivot_zones.columns.map(lambda x: f'{x[1]} {x[0]}')#merge the two index level names
pivot_zones.columns = pivot_zones.columns.map(lambda x: x[1:])##remove leading space
pivot_zones

,player_id,game_id,5-1 made,5-2 made,6-1 made,6-2 made,6-3 made,7-1 made,7-2 made,7-3 made,5-1 attempted,5-2 attempted,6-1 attempted,6-2 attempted,6-3 attempted,7-1 attempted,7-2 attempted,7-3 attempted
0,618bf7f3815098ca6660f334,636d95e21f186408f6c8d127,NaN,NaN,NaN,1.0,NaN,1.0,2.0,2.0,NaN,NaN,NaN,1.0,NaN,1.0,2.0,2.0
1,618bf7f3815098ca6660f334,636f9e5827131d1edcad44ed,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0
2,618bf7f3815098ca6660f334,637310726c21c0a3cfa28625,NaN,0.0,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,4.0,NaN
3,618bf7f3815098ca6660f334,6378674f4650d0358704664d,NaN,1.0,1.0,NaN,3.0,NaN,0.0,NaN,NaN,1.0,1.0,NaN,4.0,NaN,1.0,NaN
4,618bf7f3815098ca6660f334,637c5431fdedb41779d81fcc,NaN,1.0,NaN,0.0,NaN,1.0,2.0,0.0,NaN,2.0,NaN,1.0,NaN,2.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78511,640004f5effffd352c7ea68e,640004fbeffffd352c7eab2e,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
78512,640004f5effffd352c7ea68f,640004fbeffffd352c7eab2e,NaN,NaN,1.0,NaN,NaN,NaN,1.0,0.0,NaN,NaN,1.0,NaN,NaN,NaN,4.0,1.0
78513,640004f5effffd352c7ea690,640004fbeffffd352c7eab2e,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
78514,64018833f25daee41501b666,64018839f25daee41501ba68,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
three_pointers_df = pd.merge(three_pointers_df, pivot_zones, how= 'left', on=['player_id','game_id']).fillna(0)
three_pointers_df

,team_id,player_id,game_id,games played since,played,3PM,3PA,3P Jumpshots made,3P Jumpshots attempted,3P Pullups made,3P Pullups attempted,3P Step Backs made,3P Step Backs attempted,3P Jumpshots assisted on,5-1 made,5-2 made,6-1 made,6-2 made,6-3 made,7-1 made,7-2 made,7-3 made,5-1 attempted,5-2 attempted,6-1 attempted,6-2 attempted,6-3 attempted,7-1 attempted,7-2 attempted,7-3 attempted
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6415d5be4159b36f592a3e36,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,640c4a8c324d0dcc0472dfb1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6404dda8dde332ca06683eb8,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,64002f6baf005c7ca66db18f,4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152187,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,637af9ab6038cd4aeeba6a85,29,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152188,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6378668a4650d035870422c6,30,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152189,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,63747f5ddd66a09150111a84,31,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152190,5e875e684dc25ebee5af67d3,63a3812724a0d9396717acba,6371116581b591dcab76de28,32,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
three_pointers_df.columns[4:].to_list()

['played',
 '3PM',
 '3PA',
 '3P Jumpshots made',
 '3P Jumpshots attempted',
 '3P Pullups made',
 '3P Pullups attempted',
 '3P Step Backs made',
 '3P Step Backs attempted',
 '3P Jumpshots assisted on',
 '5-1 made',
 '5-2 made',
 '6-1 made',
 '6-2 made',
 '6-3 made',
 '7-1 made',
 '7-2 made',
 '7-3 made',
 '5-1 attempted',
 '5-2 attempted',
 '6-1 attempted',
 '6-2 attempted',
 '6-3 attempted',
 '7-1 attempted',
 '7-2 attempted',
 '7-3 attempted']

In [93]:
#Create a new dataframe for the last 1, 3, 5, 10 games, plus the season total
#Use the rolling function to compute the last x games sums, NOT INCLUDING the game in each row (closed='left')
Last_game_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(closed='left',window=1).sum()
Last_3_games_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(closed='left',window=3).sum()
Last_5_games_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(closed='left',window=5).sum()
Last_10_games_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(closed='left',window=10).sum()
Season_3P = three_pointers_df.sort_values(['team_id','player_id','games played since'], ascending=False).groupby(
    ['team_id','player_id'],as_index=False)[three_pointers_df.columns[4:].to_list()].rolling(
        closed='left',window=most_team_games, min_periods=1).sum()

In [94]:
#For each dataframe and each shot/zone type compute the respective percentages
for df in [Last_game_3P, Last_3_games_3P, Last_5_games_3P, Last_10_games_3P, Season_3P]:
    df.insert(5, '3P%', df['3PM']/df['3PA'])
    df.insert(8, 'Jumpshots 3P%', df['3P Jumpshots made']/df['3P Jumpshots attempted'])
    df.insert(11, 'Pullups 3P%', df['3P Pullups made']/df['3P Pullups attempted'])
    df.insert(14, 'Step Backs 3P%', df['3P Step Backs made']/df['3P Step Backs attempted'])
    df.insert(16, '3P Jumpshots assisted%', df['3P Jumpshots assisted on']/df['3PM'])
    df['zone 5-1 FG%'] = df['5-1 made']/df['5-1 attempted']
    df['zone 5-2 FG%'] = df['5-2 made']/df['5-2 attempted']
    df['zone 6-1 FG%'] = df['6-1 made']/df['6-1 attempted']
    df['zone 6-2 FG%'] = df['6-2 made']/df['6-2 attempted']
    df['zone 6-3 FG%'] = df['6-3 made']/df['6-3 attempted']
    df['zone 7-1 FG%'] = df['7-1 made']/df['7-1 attempted']
    df['zone 7-2 FG%'] = df['7-2 made']/df['7-2 attempted']
    df['zone 7-3 FG%'] = df['7-3 made']/df['7-3 attempted']

In [95]:
#For each dataframe and each shot/zone type normalize the respective makes and attempts
for df in [Last_game_3P, Last_3_games_3P, Last_5_games_3P, Last_10_games_3P, Season_3P]:
    df['3PM'] = np.where(df.played.isna()==True, df['3PM'], df['3PM']/df.played)
    df['3PA'] = np.where(df.played.isna()==True, df['3PA'], df['3PA']/df.played)
    df['3P Jumpshots made'] = np.where(df.played.isna()==True, df['3P Jumpshots made'], df['3P Jumpshots made']/df.played)
    df['3P Jumpshots attempted'] = np.where(df.played.isna()==True, df['3P Jumpshots attempted'], df['3P Jumpshots attempted']/df.played)
    df['3P Pullups made'] = np.where(df.played.isna()==True, df['3P Pullups made'], df['3P Pullups made']/df.played)
    df['3P Pullups attempted'] = np.where(df.played.isna()==True, df['3P Pullups attempted'], df['3P Pullups attempted']/df.played)
    df['3P Step Backs made'] = np.where(df.played.isna()==True, df['3P Step Backs made'], df['3P Step Backs made']/df.played)
    df['3P Step Backs attempted'] = np.where(df.played.isna()==True, df['3P Step Backs attempted'], df['3P Step Backs attempted']/df.played)
    df['5-1 made'] = np.where(df.played.isna()==True, df['5-1 made'], df['5-1 made']/df.played)
    df['5-1 attempted'] = np.where(df.played.isna()==True, df['5-1 attempted'], df['5-1 attempted']/df.played)
    df['5-2 made'] = np.where(df.played.isna()==True, df['5-2 made'], df['5-2 made']/df.played)
    df['5-2 attempted'] = np.where(df.played.isna()==True, df['5-2 attempted'], df['5-2 attempted']/df.played)
    df['6-1 made'] = np.where(df.played.isna()==True, df['6-1 made'], df['6-1 made']/df.played)
    df['6-1 attempted'] = np.where(df.played.isna()==True, df['6-1 attempted'], df['6-1 attempted']/df.played)
    df['6-2 made'] = np.where(df.played.isna()==True, df['6-2 made'], df['6-2 made']/df.played)
    df['6-2 attempted'] = np.where(df.played.isna()==True, df['6-2 attempted'], df['6-2 attempted']/df.played)
    df['6-3 made'] = np.where(df.played.isna()==True, df['6-3 made'], df['6-3 made']/df.played)
    df['6-3 attempted'] = np.where(df.played.isna()==True, df['6-3 attempted'], df['6-3 attempted']/df.played)
    df['7-1 made'] = np.where(df.played.isna()==True, df['7-1 made'], df['7-1 made']/df.played)
    df['7-1 attempted'] = np.where(df.played.isna()==True, df['7-1 attempted'], df['7-1 attempted']/df.played)
    df['7-2 made'] = np.where(df.played.isna()==True, df['7-2 made'], df['7-2 made']/df.played)
    df['7-2 attempted'] = np.where(df.played.isna()==True, df['7-2 attempted'], df['7-2 attempted']/df.played)
    df['7-3 made'] = np.where(df.played.isna()==True, df['7-3 made'], df['7-3 made']/df.played)
    df['7-3 attempted'] = np.where(df.played.isna()==True, df['7-3 attempted'], df['7-3 attempted']/df.played)

In [96]:
#Rename the columns before merging all the dataframes together
Last_game_3P.columns = Last_game_3P.columns.map(lambda x: f'L1 {x}')
Last_3_games_3P.columns = Last_3_games_3P.columns.map(lambda x: f'L3 {x}')
Last_5_games_3P.columns = Last_5_games_3P.columns.map(lambda x: f'L5 {x}')
Last_10_games_3P.columns = Last_10_games_3P.columns.map(lambda x: f'L10 {x}')
Season_3P.columns = Season_3P.columns.map(lambda x: f'TOT {x}')
Last_game_3P = Last_game_3P.rename(columns={'L1 team_id':'team_id','L1 player_id':'player_id'})
Last_3_games_3P = Last_3_games_3P.rename(columns={'L3 team_id':'team_id','L3 player_id':'player_id'})
Last_5_games_3P = Last_5_games_3P.rename(columns={'L5 team_id':'team_id','L5 player_id':'player_id'})
Last_10_games_3P = Last_10_games_3P.rename(columns={'L10 team_id':'team_id','L10 player_id':'player_id'})
Season_3P = Season_3P.rename(columns={'TOT team_id':'team_id','TOT player_id':'player_id'})

In [97]:
#Concatenate all the dataframes into one
feature_df_3P = pd.concat([team_player_game_df, Last_game_3P.iloc[:,2:], 
                           Last_3_games_3P.iloc[:,2:], Last_5_games_3P.iloc[:,2:], 
                           Last_10_games_3P.iloc[:,2:], Season_3P.iloc[:,2:]], axis=1)
feature_df_3P.to_pickle("3P_features_df.pickle")#write the data to a pickle file
feature_df_3P

,team_id,player_id,game_id,games played since,played,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots made,L1 3P Jumpshots attempted,L1 Jumpshots 3P%,L1 3P Pullups made,L1 3P Pullups attempted,L1 Pullups 3P%,L1 3P Step Backs made,L1 3P Step Backs attempted,L1 Step Backs 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L1 5-1 made,L1 5-2 made,L1 6-1 made,L1 6-2 made,L1 6-3 made,L1 7-1 made,L1 7-2 made,L1 7-3 made,L1 5-1 attempted,L1 5-2 attempted,L1 6-1 attempted,L1 6-2 attempted,L1 6-3 attempted,L1 7-1 attempted,L1 7-2 attempted,L1 7-3 attempted,L1 zone 5-1 FG%,L1 zone 5-2 FG%,L1 zone 6-1 FG%,L1 zone 6-2 FG%,L1 zone 6-3 FG%,L1 zone 7-1 FG%,L1 zone 7-2 FG%,L1 zone 7-3 FG%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots made,L3 3P Jumpshots attempted,L3 Jumpshots 3P%,L3 3P Pullups made,L3 3P Pullups attempted,L3 Pullups 3P%,L3 3P Step Backs made,L3 3P Step Backs attempted,L3 Step Backs 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L3 5-1 made,L3 5-2 made,L3 6-1 made,L3 6-2 made,L3 6-3 made,L3 7-1 made,L3 7-2 made,L3 7-3 made,L3 5-1 attempted,L3 5-2 attempted,L3 6-1 attempted,L3 6-2 attempted,L3 6-3 attempted,L3 7-1 attempted,L3 7-2 attempted,L3 7-3 attempted,L3 zone 5-1 FG%,L3 zone 5-2 FG%,L3 zone 6-1 FG%,L3 zone 6-2 FG%,L3 zone 6-3 FG%,L3 zone 7-1 FG%,L3 zone 7-2 FG%,L3 zone 7-3 FG%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots made,L5 3P Jumpshots attempted,L5 Jumpshots 3P%,L5 3P Pullups made,L5 3P Pullups attempted,L5 Pullups 3P%,L5 3P Step Backs made,L5 3P Step Backs attempted,L5 Step Backs 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L5 5-1 made,L5 5-2 made,L5 6-1 made,L5 6-2 made,L5 6-3 made,L5 7-1 made,L5 7-2 made,L5 7-3 made,L5 5-1 attempted,L5 5-2 attempted,L5 6-1 attempted,L5 6-2 attempted,L5 6-3 attempted,L5 7-1 attempted,L5 7-2 attempted,L5 7-3 attempted,L5 zone 5-1 FG%,L5 zone 5-2 FG%,L5 zone 6-1 FG%,L5 zone 6-2 FG%,L5 zone 6-3 FG%,L5 zone 7-1 FG%,L5 zone 7-2 FG%,L5 zone 7-3 FG%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots made,L10 3P Jumpshots attempted,L10 Jumpshots 3P%,L10 3P Pullups made,L10 3P Pullups attempted,L10 Pullups 3P%,L10 3P Step Backs made,L10 3P Step Backs attempted,L10 Step Backs 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,L10 5-1 made,L10 5-2 made,L10 6-1 made,L10 6-2 made,L10 6-3 made,L10 7-1 made,L10 7-2 made,L10 7-3 made,L10 5-1 attempted,L10 5-2 attempted,L10 6-1 attempted,L10 6-2 attempted,L10 6-3 attempted,L10 7-1 attempted,L10 7-2 attempted,L10 7-3 attempted,L10 zone 5-1 FG%,L10 zone 5-2 FG%,L10 zone 6-1 FG%,L10 zone 6-2 FG%,L10 zone 6-3 FG%,L10 zone 7-1 FG%,L10 zone 7-2 FG%,L10 zone 7-3 FG%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots made,TOT 3P Jumpshots attempted,TOT Jumpshots 3P%,TOT 3P Pullups made,TOT 3P Pullups attempted,TOT Pullups 3P%,TOT 3P Step Backs made,TOT 3P Step Backs attempted,TOT Step Backs 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,TOT 5-1 made,TOT 5-2 made,TOT 6-1 made,TOT 6-2 made,TOT 6-3 made,TOT 7-1 made,TOT 7-2 made,TOT 7-3 made,TOT 5-1 attempted,TOT 5-2 attempted,TOT 6-1 attempted,TOT 6-2 attempted,TOT 6-3 attempted,TOT 7-1 attempted,TOT 7-2 attempted,TOT 7-3 attempted,TOT zone 5-1 FG%,TOT zone 5-2 FG%,TOT zone 6-1 FG%,TOT zone 6-2 FG%,TOT zone 6-3 FG%,TOT zone 7-1 FG%,TOT zone 7-2 FG%,TOT zone 7-3 FG%
0,5e875e684dc25ebee5af6b87,618ff2add1901c63278a2703,6418672cc9f923fdd1fb564d,0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [98]:
#Now that current season's features are created, we will merge them with last season's features
Last_season_3P_features_df

,player_id,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots made,Last_season 3P Jumpshots attempted,Last_season Jumpshots 3P%,Last_season 3P Pullups made,Last_season 3P Pullups attempted,Last_season Pullups 3P%,Last_season 3P Step Backs made,Last_season 3P Step Backs attempted,Last_season Step Backs 3P%,Last_season 3P Jumpshots assisted on,Last_season 5-1 made,Last_season 3P Jumpshots assisted%,Last_season 5-2 made,Last_season 6-1 made,Last_season 6-2 made,Last_season 6-3 made,Last_season 7-1 made,Last_season 7-2 made,Last_season 7-3 made,Last_season 5-1 attempted,Last_season 5-2 attempted,Last_season 6-1 attempted,Last_season 6-2 attempted,Last_season 6-3 attempted,Last_season 7-1 attempted,Last_season 7-2 attempted,Last_season 7-3 attempted,Last_season zone 5-1 FG%,Last_season zone 5-2 FG%,Last_season zone 6-1 FG%,Last_season zone 6-2 FG%,Last_season zone 6-3 FG%,Last_season zone 7-1 FG%,Last_season zone 7-2 FG%,Last_season zone 7-3 FG%
0,618da14c4db298feb94be0d1,24,1.166667,3.666667,0.318182,1.166667,3.666667,0.318182,0.0,0.0,NaN,0.000000,0.000000,NaN,27.0,0.125000,0.964286,0.041667,0.166667,0.208333,0.291667,0.000000,0.250000,0.083333,0.375000,0.583333,0.541667,0.541667,0.750000,0.041667,0.708333,0.125000,0.333333,0.071429,0.307692,0.384615,0.388889,0.0,0.352941,0.666667
1,618da14c4db298feb94be0d2,24,0.000000,0.041667,0.000000,0.000000,0.041667,0.000000,0.0,0.0,NaN,0.000000,0.000000,NaN,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
2,618da14c4db298feb94be0d3,23,0.826087,2.608696,0.316667,0.739130,2.521739,0.293103,0.0,0.0,NaN,0.043478,0.043478,1.0,17.0,0.086957,0.894737,0.000000,0.260870,0.130435,0.043478,0.000000,0.304348,0.000000,0.260870,0.130435,0.565217,0.652174,0.260870,0.130435,0.608696,0.000000,0.333333,0.000000,0.461538,0.200000,0.166667,0.0,0.500000,NaN
3,618da14c4db298feb94be0d4,24,1.000000,3.041667,0.328767,0.958333,3.000000,0.319444,0.0,0.0,NaN,0.041667,0.041667,1.0,16.0,0.166667,0.666667,0.041667,0.208333,0.291667,0.041667,0.083333,0.125000,0.041667,0.208333,0.083333,0.208333,0.916667,0.250000,0.083333,1.000000,0.291667,0.800000,0.500000,1.000000,0.318182,0.166667,1.0,0.125000,0.142857
4,618da14c4db298feb94be0d5,22,0.863636,2.681818,0.322034,0.863636,2.681818,0.322034,0.0,0.0,NaN,0.000000,0.000000,NaN,18.0,0.045455,0.947368,0.136364,0.136364,0.181818,0.090909,0.045455,0.181818,0.045455,0.136364,0.227273,0.454545,0.590909,0.545455,0.045455,0.545455,0.136364,0.333333,0.600000,0.300000,0.307692,0.166667,1.0,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,6192aaef73c0a0725fd808cc,12,0.083333,0.416667,0.200000,0.083333,0.416667,0.200000,0.0,0.0,NaN,0.000000,0.000000,NaN,1.0,0.083333,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.083333,0.000000,0.000000,0.500000,NaN,NaN,0.000000,NaN,0.0,NaN,NaN
4395,6255a71bed9cbb2fe0fe5d07,18,0.111111,0.944444,0.117647,0.111111,0.944444,0.117647,0.0,0.0,NaN,0.000000,0.000000,NaN,2.0,0.055556,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.111111,0.055556,0.277778,0.166667,0.111111,0.055556,0.055556,0.111111,0.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.500000
4396,6255a71bed9cbb2fe0fe5d08,7,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.0,0.0,NaN,0.000000,0.000000,NaN,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4397,6255cd5d2c06a1d633649449,2,0.000000,0.500000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,NaN,0.000000,0.000000,NaN,0.0,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [99]:
Threes_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1
0,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c3425dd,3,2,1,636a5f91531336e79c3425b4,18:50,1130,ThreeMake,7-1,Three,1,5.0,20.7,37.7,25.26,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
6,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe45,636a5fa6531336e79c3425fc,7,12,1,636a5f91531336e79c3425b7,13:6,786,ThreeMiss,7-2,Three,0,33.5,31.0,108.3,27.14,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
8,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c342610,11,17,1,636a5f91531336e79c3425b9,10:51,651,ThreeMiss,6-3,Three,0,41.5,23.5,132.1,24.60,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
9,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c342616,11,19,1,636a5f91531336e79c3425ba,10:6,606,ThreeMiss,7-2,Three,0,26.0,32.0,92.1,26.73,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
11,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c34261a,11,21,1,636a5f91531336e79c3425ba,9:25,565,ThreeMiss,5-2,Three,0,49.0,7.5,174.6,24.11,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698285,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab19a32a50e3d7afdc0,648b3a7b98debb5769861cbf,73,69,2,648b3a0b98debb576986198a,2:48,168,ThreeMake,6-2,Three,1,25.5,29.1,91.2,23.90,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
698287,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab19a32a50e3d7afdc0,648b3a7b98debb5769861cc3,73,73,2,648b3a0b98debb576986198a,1:58,118,ThreeMiss,6-2,Three,0,37.5,24.4,123.1,22.90,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
698292,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,61e5d81b335e7288b5d1286e,648b3a7b98debb5769861ccd,80,82,3,648b3a0b98debb576986198b,3:26,206,ThreeMake,6-2,Three,1,10.5,25.4,54.2,24.81,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
698293,648b3a9e98debb5769861e35,5e875e684dc25ebee5af6b7b,63502c8a4e970ea2c1102c01,618dbab19a32a50e3d7afdc2,648b3a7b98debb5769861cdb,83,85,3,648b3a0b98debb576986198e,1:52,112,ThreeMake,5-2,Three,1,48.0,8.5,172.1,23.22,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [100]:
reduced_3P_df = pd.merge(Threes_df, feature_df_3P, how='left', on=['team_id','player_id','game_id'])
reduced_3P_df = pd.merge(reduced_3P_df, Last_season_3P_features_df, how='left', on=['player_id'])
reduced_3P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots made,L1 3P Jumpshots attempted,L1 Jumpshots 3P%,L1 3P Pullups made,L1 3P Pullups attempted,L1 Pullups 3P%,L1 3P Step Backs made,L1 3P Step Backs attempted,L1 Step Backs 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L1 5-1 made,L1 5-2 made,L1 6-1 made,L1 6-2 made,L1 6-3 made,L1 7-1 made,L1 7-2 made,L1 7-3 made,L1 5-1 attempted,L1 5-2 attempted,L1 6-1 attempted,L1 6-2 attempted,L1 6-3 attempted,L1 7-1 attempted,L1 7-2 attempted,L1 7-3 attempted,L1 zone 5-1 FG%,L1 zone 5-2 FG%,L1 zone 6-1 FG%,L1 zone 6-2 FG%,L1 zone 6-3 FG%,L1 zone 7-1 FG%,L1 zone 7-2 FG%,L1 zone 7-3 FG%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots made,L3 3P Jumpshots attempted,L3 Jumpshots 3P%,L3 3P Pullups made,L3 3P Pullups attempted,L3 Pullups 3P%,L3 3P Step Backs made,L3 3P Step Backs attempted,L3 Step Backs 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L3 5-1 made,L3 5-2 made,L3 6-1 made,L3 6-2 made,L3 6-3 made,L3 7-1 made,L3 7-2 made,L3 7-3 made,L3 5-1 attempted,L3 5-2 attempted,L3 6-1 attempted,L3 6-2 attempted,L3 6-3 attempted,L3 7-1 attempted,L3 7-2 attempted,L3 7-3 attempted,L3 zone 5-1 FG%,L3 zone 5-2 FG%,L3 zone 6-1 FG%,L3 zone 6-2 FG%,L3 zone 6-3 FG%,L3 zone 7-1 FG%,L3 zone 7-2 FG%,L3 zone 7-3 FG%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots made,L5 3P Jumpshots attempted,L5 Jumpshots 3P%,L5 3P Pullups made,L5 3P Pullups attempted,L5 Pullups 3P%,L5 3P Step Backs made,L5 3P Step Backs attempted,L5 Step Backs 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L5 5-1 made,L5 5-2 made,L5 6-1 made,L5 6-2 made,L5 6-3 made,L5 7-1 made,L5 7-2 made,L5 7-3 made,L5 5-1 attempted,L5 5-2 attempted,L5 6-1 attempted,L5 6-2 attempted,L5 6-3 attempted,L5 7-1 attempted,L5 7-2 attempted,L5 7-3 attempted,L5 zone 5-1 FG%,L5 zone 5-2 FG%,L5 zone 6-1 FG%,L5 zone 6-2 FG%,L5 zone 6-3 FG%,L5 zone 7-1 FG%,L5 zone 7-2 FG%,L5 zone 7-3 FG%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots made,L10 3P Jumpshots attempted,L10 Jumpshots 3P%,L10 3P Pullups made,L10 3P Pullups attempted,L10 Pullups 3P%,L10 3P Step Backs made,L10 3P Step Backs attempted,L10 Step Backs 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,L10 5-1 made,L10 5-2 made,L10 6-1 made,L10 6-2 made,L10 6-3 made,L10 7-1 made,L10 7-2 made,L10 7-3 made,L10 5-1 attempted,L10 5-2 attempted,L10 6-1 attempted,L10 6-2 attempted,L10 6-3 attempted,L10 7-1 attempted,L10 7-2 attempted,L10 7-3 attempted,L10 zone 5-1 FG%,L10 zone 5-2 FG%,L10 zone 6-1 FG%,L10 zone 6-2 FG%,L10 zone 6-3 FG%,L10 zone 7-1 FG%,L10 zone 7-2 FG%,L10 zone 7-3 FG%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots made,TOT 3P Jumpshots attempted,TOT Jumpshots 3P%,TOT 3P Pullups made,TOT 3P Pullups attempted,TOT Pullups 3P%,TOT 3P Step Backs made,TOT 3P Step Backs attempted,TOT Step Backs 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,TOT 5-1 made,TOT 5-2 made,TOT 6-1 made,TOT 6-2 made,TOT 6-3 made,TOT 7-1 made,TOT 7-2 made,TOT 7-3 made,TOT 5-1 attempted,TOT 5-2 attempted,TOT 6-1 attempted,TOT 6-2 attempted,TOT 6-3 attempted,TOT 7-1 attempted,TOT 7-2 attempted,TOT 7-3 attempted,TOT zone 5-1 FG%,TOT zone 5-2 FG%,TOT zone 6-1 FG%,TOT zone 6-2 FG%,TOT zone 6-3 FG%,TOT zone 7-1 FG%,TOT zone 7-2 FG%,TOT zone 7-3 FG%,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots made,Last_season 3P Jumpshots attempted,Last_season Jumpshots 3P%,Last_season 3P Pullups made,Last_season 3P Pullups attempted,Last_season Pullups 3P%,Last_season 3P Step Backs made,Last_season 3P Step Backs attempted,Last_season Step Backs 3P%,Last_season 3P Jumpshots assisted on,Last_season 5-1 made,Last_sea

In [101]:
NAs3 = reduced_3P_df.isna().sum()/reduced_3P_df.shape[0]

In [102]:
print(NAs3[42::39])

L1 3PM             0.167526
L3 3PM             0.197640
L5 3PM             0.248917
L10 3PM            0.386211
TOT 3PM            0.135233
Last_season 3PM    0.327311
dtype: float64


In [103]:
reduced_3P_df.insert(41, 'Last_season Zone perc%', np.nan)
reduced_3P_df.insert(41, 'Last_season Zone attempted', 0)
reduced_3P_df.insert(41, 'Last_season Zone made', 0)
reduced_3P_df.insert(41, 'TOT Zone perc%', np.nan)
reduced_3P_df.insert(41, 'TOT Zone attempted', 0)
reduced_3P_df.insert(41, 'TOT Zone made', 0)
reduced_3P_df.insert(41, 'L10 Zone perc%', np.nan)
reduced_3P_df.insert(41, 'L10 Zone attempted', 0)
reduced_3P_df.insert(41, 'L10 Zone made', 0)
reduced_3P_df.insert(41, 'L5 Zone perc%', np.nan)
reduced_3P_df.insert(41, 'L5 Zone attempted', 0)
reduced_3P_df.insert(41, 'L5 Zone made', 0)
reduced_3P_df.insert(41, 'L3 Zone perc%', np.nan)
reduced_3P_df.insert(41, 'L3 Zone attempted', 0)
reduced_3P_df.insert(41, 'L3 Zone made', 0)
reduced_3P_df.insert(41, 'L1 Zone perc%', np.nan)
reduced_3P_df.insert(41, 'L1 Zone attempted', 0)
reduced_3P_df.insert(41, 'L1 Zone made', 0)
reduced_3P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots made,L1 3P Jumpshots attempted,L1 Jumpshots 3P%,L1 3P Pullups made,L1 3P Pullups attempted,L1 Pullups 3P%,L1 3P Step Backs made,L1 3P Step Backs attempted,L1 Step Backs 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L1 5-1 made,L1 5-2 made,L1 6-1 made,L1 6-2 made,L1 6-3 made,L1 7-1 made,L1 7-2 made,L1 7-3 made,L1 5-1 attempted,L1 5-2 attempted,L1 6-1 attempted,L1 6-2 attempted,L1 6-3 attempted,L1 7-1 attempted,L1 7-2 attempted,L1 7-3 attempted,L1 zone 5-1 FG%,L1 zone 5-2 FG%,L1 zone 6-1 FG%,L1 zone 6-2 FG%,L1 zone 6-3 FG%,L1 zone 7-1 FG%,L1 zone 7-2 FG%,L1 zone 7-3 FG%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots made,L3 3P Jumpshots attempted,L3 Jumpshots 3P%,L3 3P Pullups made,L3 3P Pullups attempted,L3 Pullups 3P%,L3 3P Step Backs made,L3 3P Step Backs attempted,L3 Step Backs 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L3 5-1 made,L3 5-2 made,L3 6-1 made,L3 6-2 made,L3 6-3 made,L3 7-1 made,L3 7-2 made,L3 7-3 made,L3 5-1 attempted,L3 5-2 attempted,L3 6-1 attempted,L3 6-2 attempted,L3 6-3 attempted,L3 7-1 attempted,L3 7-2 attempted,L3 7-3 attempted,L3 zone 5-1 FG%,L3 zone 5-2 FG%,L3 zone 6-1 FG%,L3 zone 6-2 FG%,L3 zone 6-3 FG%,L3 zone 7-1 FG%,L3 zone 7-2 FG%,L3 zone 7-3 FG%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots made,L5 3P Jumpshots attempted,L5 Jumpshots 3P%,L5 3P Pullups made,L5 3P Pullups attempted,L5 Pullups 3P%,L5 3P Step Backs made,L5 3P Step Backs attempted,L5 Step Backs 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L5 5-1 made,L5 5-2 made,L5 6-1 made,L5 6-2 made,L5 6-3 made,L5 7-1 made,L5 7-2 made,L5 7-3 made,L5 5-1 attempted,L5 5-2 attempted,L5 6-1 attempted,L5 6-2 attempted,L5 6-3 attempted,L5 7-1 attempted,L5 7-2 attempted,L5 7-3 attempted,L5 zone 5-1 FG%,L5 zone 5-2 FG%,L5 zone 6-1 FG%,L5 zone 6-2 FG%,L5 zone 6-3 FG%,L5 zone 7-1 FG%,L5 zone 7-2 FG%,L5 zone 7-3 FG%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots made,L10 3P Jumpshots attempted,L10 Jumpshots 3P%,L10 3P Pullups made,L10 3P Pullups attempted,L10 Pullups 3P%,L10 3P Step Backs made,L10 3P Step Backs attempted,L10 Step Backs 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,L10 5-1 made,L10 5-2 made,L10 6-1 made,L10 6-2 made,L10 6-3 made,L10 7-1 made,L10 7-2 made,L10 7-3 made,L10 5-1 attempted,L10 5-2 attempted,L10 6-1 attempted,L10 6-2 attempted,L10 6-3 attempted,L10 7-1 attempted,L10 7-2 attempted,L10 7-3 attempted,L10 zone 5-1 FG%,L10 zone 5-2 FG%,L10 zone 6-1 FG%,L10 zone 6-2 FG%,L10 zone 6-3 FG%,L10 zone 7-1 FG%,L10 zone 7-2 FG%,L10 zone 7-3 FG%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots made,TOT 3P Jumpshots attempted,TOT Jumpshots 3P%,TOT 3P Pullups made,TOT 3P Pullups attempted,TOT Pullups 3P%,TOT 3P Step Backs made,TOT 3P Step Backs attempted,TOT Step Backs 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,TOT 5-1 made,TOT 5-2 made,TOT 6-1 made,TOT 6-2 made,TOT 6-3 made,TOT 7-1 made,TOT 7-2 made,TOT 7-3 made,TOT 5-1 attempted,TOT 5-2 attempted,TOT 6-1 attempted,TOT 6-2 attempted,TOT 6-3 attempted,TOT 7-1 attempted,TOT 7-2 attempted,TOT 7-3 attempted,TOT zone 5-1 FG%,TOT zone 5-2 FG%,TOT zone 6-1 FG%,TOT zone 6-2 FG%,TOT zone 6-3 FG%,TOT zone 7-1 FG%,TOT zone 7-2 FG%,TOT zone 7-3 FG%,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots made,Las

In [104]:
cols_to_drop = list()

for time in ['L1','L3','L5','L10','TOT','Last_season']:
    for zone in [' 5-1',' 5-2',' 6-1',' 6-2',' 6-3',' 7-1',' 7-2',' 7-3']:
        reduced_3P_df[f"{time} Zone made"] = np.where(reduced_3P_df.Zone==zone, reduced_3P_df[f"{time}{zone} made"], reduced_3P_df[f"{time} Zone made"])
        cols_to_drop.append(f"{time}{zone} made")
        reduced_3P_df[f"{time} Zone attempted"] = np.where(reduced_3P_df.Zone==zone, reduced_3P_df[f"{time}{zone} attempted"], reduced_3P_df[f"{time} Zone attempted"])
        cols_to_drop.append(f"{time}{zone} attempted")
        reduced_3P_df[f"{time} Zone perc%"] = np.where(reduced_3P_df.Zone==zone, reduced_3P_df[f"{time} zone{zone} FG%"], reduced_3P_df[f"{time} Zone perc%"])
        cols_to_drop.append(f"{time} zone{zone} FG%")

reduced_3P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots made,L1 3P Jumpshots attempted,L1 Jumpshots 3P%,L1 3P Pullups made,L1 3P Pullups attempted,L1 Pullups 3P%,L1 3P Step Backs made,L1 3P Step Backs attempted,L1 Step Backs 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L1 5-1 made,L1 5-2 made,L1 6-1 made,L1 6-2 made,L1 6-3 made,L1 7-1 made,L1 7-2 made,L1 7-3 made,L1 5-1 attempted,L1 5-2 attempted,L1 6-1 attempted,L1 6-2 attempted,L1 6-3 attempted,L1 7-1 attempted,L1 7-2 attempted,L1 7-3 attempted,L1 zone 5-1 FG%,L1 zone 5-2 FG%,L1 zone 6-1 FG%,L1 zone 6-2 FG%,L1 zone 6-3 FG%,L1 zone 7-1 FG%,L1 zone 7-2 FG%,L1 zone 7-3 FG%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots made,L3 3P Jumpshots attempted,L3 Jumpshots 3P%,L3 3P Pullups made,L3 3P Pullups attempted,L3 Pullups 3P%,L3 3P Step Backs made,L3 3P Step Backs attempted,L3 Step Backs 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L3 5-1 made,L3 5-2 made,L3 6-1 made,L3 6-2 made,L3 6-3 made,L3 7-1 made,L3 7-2 made,L3 7-3 made,L3 5-1 attempted,L3 5-2 attempted,L3 6-1 attempted,L3 6-2 attempted,L3 6-3 attempted,L3 7-1 attempted,L3 7-2 attempted,L3 7-3 attempted,L3 zone 5-1 FG%,L3 zone 5-2 FG%,L3 zone 6-1 FG%,L3 zone 6-2 FG%,L3 zone 6-3 FG%,L3 zone 7-1 FG%,L3 zone 7-2 FG%,L3 zone 7-3 FG%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots made,L5 3P Jumpshots attempted,L5 Jumpshots 3P%,L5 3P Pullups made,L5 3P Pullups attempted,L5 Pullups 3P%,L5 3P Step Backs made,L5 3P Step Backs attempted,L5 Step Backs 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L5 5-1 made,L5 5-2 made,L5 6-1 made,L5 6-2 made,L5 6-3 made,L5 7-1 made,L5 7-2 made,L5 7-3 made,L5 5-1 attempted,L5 5-2 attempted,L5 6-1 attempted,L5 6-2 attempted,L5 6-3 attempted,L5 7-1 attempted,L5 7-2 attempted,L5 7-3 attempted,L5 zone 5-1 FG%,L5 zone 5-2 FG%,L5 zone 6-1 FG%,L5 zone 6-2 FG%,L5 zone 6-3 FG%,L5 zone 7-1 FG%,L5 zone 7-2 FG%,L5 zone 7-3 FG%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots made,L10 3P Jumpshots attempted,L10 Jumpshots 3P%,L10 3P Pullups made,L10 3P Pullups attempted,L10 Pullups 3P%,L10 3P Step Backs made,L10 3P Step Backs attempted,L10 Step Backs 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,L10 5-1 made,L10 5-2 made,L10 6-1 made,L10 6-2 made,L10 6-3 made,L10 7-1 made,L10 7-2 made,L10 7-3 made,L10 5-1 attempted,L10 5-2 attempted,L10 6-1 attempted,L10 6-2 attempted,L10 6-3 attempted,L10 7-1 attempted,L10 7-2 attempted,L10 7-3 attempted,L10 zone 5-1 FG%,L10 zone 5-2 FG%,L10 zone 6-1 FG%,L10 zone 6-2 FG%,L10 zone 6-3 FG%,L10 zone 7-1 FG%,L10 zone 7-2 FG%,L10 zone 7-3 FG%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots made,TOT 3P Jumpshots attempted,TOT Jumpshots 3P%,TOT 3P Pullups made,TOT 3P Pullups attempted,TOT Pullups 3P%,TOT 3P Step Backs made,TOT 3P Step Backs attempted,TOT Step Backs 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,TOT 5-1 made,TOT 5-2 made,TOT 6-1 made,TOT 6-2 made,TOT 6-3 made,TOT 7-1 made,TOT 7-2 made,TOT 7-3 made,TOT 5-1 attempted,TOT 5-2 attempted,TOT 6-1 attempted,TOT 6-2 attempted,TOT 6-3 attempted,TOT 7-1 attempted,TOT 7-2 attempted,TOT 7-3 attempted,TOT zone 5-1 FG%,TOT zone 5-2 FG%,TOT zone 6-1 FG%,TOT zone 6-2 FG%,TOT zone 6-3 FG%,TOT zone 7-1 FG%,TOT zone 7-2 FG%,TOT zone 7-3 FG%,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots made,Las

In [105]:
reduced_3P_df = reduced_3P_df.drop(columns=cols_to_drop)
reduced_3P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots made,L1 3P Jumpshots attempted,L1 Jumpshots 3P%,L1 3P Pullups made,L1 3P Pullups attempted,L1 Pullups 3P%,L1 3P Step Backs made,L1 3P Step Backs attempted,L1 Step Backs 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots made,L3 3P Jumpshots attempted,L3 Jumpshots 3P%,L3 3P Pullups made,L3 3P Pullups attempted,L3 Pullups 3P%,L3 3P Step Backs made,L3 3P Step Backs attempted,L3 Step Backs 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots made,L5 3P Jumpshots attempted,L5 Jumpshots 3P%,L5 3P Pullups made,L5 3P Pullups attempted,L5 Pullups 3P%,L5 3P Step Backs made,L5 3P Step Backs attempted,L5 Step Backs 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots made,L10 3P Jumpshots attempted,L10 Jumpshots 3P%,L10 3P Pullups made,L10 3P Pullups attempted,L10 Pullups 3P%,L10 3P Step Backs made,L10 3P Step Backs attempted,L10 Step Backs 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots made,TOT 3P Jumpshots attempted,TOT Jumpshots 3P%,TOT 3P Pullups made,TOT 3P Pullups attempted,TOT Pullups 3P%,TOT 3P Step Backs made,TOT 3P Step Backs attempted,TOT Step Backs 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots made,Last_season 3P Jumpshots attempted,Last_season Jumpshots 3P%,Last_season 3P Pullups made,Last_season 3P Pullups attempted,Last_season Pullups 3P%,Last_season 3P Step Backs made,Last_season 3P Step Backs attempted,Last_season Step Backs 3P%,Last_season 3P Jumpshots assisted on,Last_season 3P Jumpshots assisted%
0,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c3425dd,3,2,1,636a5f91531336e79c3425b4,18:50,1130,ThreeMake,7-1,Three,1,5.0,20.7,37.7,25.26,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.035714,0.035714,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,1.071429,3.214286,0.333333,0.000000,0.000000,NaN,0.000000,0.000000,NaN,28.0,0.933333
1,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe45,636a5fa6531336e79c3425fc,7,12,1,636a5f91531336e79c3425b7,13:6,786,ThreeMiss,7-2,Three,0,33.5,31.0,108.3,27.14,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [106]:
reduced_3P_df.insert(59, 'Shot type', 'Other')
reduced_3P_df['Shot type'] = np.where(reduced_3P_df.Jumpshot==True, 'Jumpshot', reduced_3P_df['Shot type'])
reduced_3P_df['Shot type'] = np.where(reduced_3P_df.Pullup==True, 'Pullup', reduced_3P_df['Shot type'])
reduced_3P_df['Shot type'] = np.where(reduced_3P_df['Step Back']==True, 'Step Back', reduced_3P_df['Shot type'])
reduced_3P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,Shot type,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots made,L1 3P Jumpshots attempted,L1 Jumpshots 3P%,L1 3P Pullups made,L1 3P Pullups attempted,L1 Pullups 3P%,L1 3P Step Backs made,L1 3P Step Backs attempted,L1 Step Backs 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots made,L3 3P Jumpshots attempted,L3 Jumpshots 3P%,L3 3P Pullups made,L3 3P Pullups attempted,L3 Pullups 3P%,L3 3P Step Backs made,L3 3P Step Backs attempted,L3 Step Backs 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots made,L5 3P Jumpshots attempted,L5 Jumpshots 3P%,L5 3P Pullups made,L5 3P Pullups attempted,L5 Pullups 3P%,L5 3P Step Backs made,L5 3P Step Backs attempted,L5 Step Backs 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots made,L10 3P Jumpshots attempted,L10 Jumpshots 3P%,L10 3P Pullups made,L10 3P Pullups attempted,L10 Pullups 3P%,L10 3P Step Backs made,L10 3P Step Backs attempted,L10 Step Backs 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots made,TOT 3P Jumpshots attempted,TOT Jumpshots 3P%,TOT 3P Pullups made,TOT 3P Pullups attempted,TOT Pullups 3P%,TOT 3P Step Backs made,TOT 3P Step Backs attempted,TOT Step Backs 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots made,Last_season 3P Jumpshots attempted,Last_season Jumpshots 3P%,Last_season 3P Pullups made,Last_season 3P Pullups attempted,Last_season Pullups 3P%,Last_season 3P Step Backs made,Last_season 3P Step Backs attempted,Last_season Step Backs 3P%,Last_season 3P Jumpshots assisted on,Last_season 3P Jumpshots assisted%
0,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c3425dd,3,2,1,636a5f91531336e79c3425b4,18:50,1130,ThreeMake,7-1,Three,1,5.0,20.7,37.7,25.26,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.035714,0.035714,1.000000,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,1.071429,3.214286,0.333333,0.000000,0.000000,NaN,0.000000,0.000000,NaN,28.0,0.933333
1,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe45,636a5fa6531336e79c3425fc,7,12,1,636a5f91531336e79c3425b7,13:6,786,ThreeMiss,7-2,Three,0,33.5,31.0,108.3,27.14,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.000000,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [107]:
reduced_3P_df['Shot type'].value_counts()

Jumpshot     250246
Pullup         7090
Step Back      4537
Other            98
Name: Shot type, dtype: int64

In [108]:
reduced_3P_df.insert(60, 'Last_season Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'Last_season Shot type attempted', 0)
reduced_3P_df.insert(60, 'Last_season Shot type made', 0)
reduced_3P_df.insert(60, 'TOT Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'TOT Shot type attempted', 0)
reduced_3P_df.insert(60, 'TOT Shot type made', 0)
reduced_3P_df.insert(60, 'L10 Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'L10 Shot type attempted', 0)
reduced_3P_df.insert(60, 'L10 Shot type made', 0)
reduced_3P_df.insert(60, 'L5 Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'L5 Shot type attempted', 0)
reduced_3P_df.insert(60, 'L5 Shot type made', 0)
reduced_3P_df.insert(60, 'L3 Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'L3 Shot type attempted', 0)
reduced_3P_df.insert(60, 'L3 Shot type made', 0)
reduced_3P_df.insert(60, 'L1 Shot type perc%', np.nan)
reduced_3P_df.insert(60, 'L1 Shot type attempted', 0)
reduced_3P_df.insert(60, 'L1 Shot type made', 0)
reduced_3P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,Shot type,L1 Shot type made,L1 Shot type attempted,L1 Shot type perc%,L3 Shot type made,L3 Shot type attempted,L3 Shot type perc%,L5 Shot type made,L5 Shot type attempted,L5 Shot type perc%,L10 Shot type made,L10 Shot type attempted,L10 Shot type perc%,TOT Shot type made,TOT Shot type attempted,TOT Shot type perc%,Last_season Shot type made,Last_season Shot type attempted,Last_season Shot type perc%,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots made,L1 3P Jumpshots attempted,L1 Jumpshots 3P%,L1 3P Pullups made,L1 3P Pullups attempted,L1 Pullups 3P%,L1 3P Step Backs made,L1 3P Step Backs attempted,L1 Step Backs 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots made,L3 3P Jumpshots attempted,L3 Jumpshots 3P%,L3 3P Pullups made,L3 3P Pullups attempted,L3 Pullups 3P%,L3 3P Step Backs made,L3 3P Step Backs attempted,L3 Step Backs 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots made,L5 3P Jumpshots attempted,L5 Jumpshots 3P%,L5 3P Pullups made,L5 3P Pullups attempted,L5 Pullups 3P%,L5 3P Step Backs made,L5 3P Step Backs attempted,L5 Step Backs 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots made,L10 3P Jumpshots attempted,L10 Jumpshots 3P%,L10 3P Pullups made,L10 3P Pullups attempted,L10 Pullups 3P%,L10 3P Step Backs made,L10 3P Step Backs attempted,L10 Step Backs 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots made,TOT 3P Jumpshots attempted,TOT Jumpshots 3P%,TOT 3P Pullups made,TOT 3P Pullups attempted,TOT Pullups 3P%,TOT 3P Step Backs made,TOT 3P Step Backs attempted,TOT Step Backs 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots made,Last_season 3P Jumpshots attempted,Last_season Jumpshots 3P%,Last_season 3P Pullups made,Last_season 3P Pullups attempted,Last_season Pullups 3P%,Last_season 3P Step Backs made,Last_season 3P Step Backs attempted,Last_season Step Backs 3P%,Last_season 3P Jumpshots assisted on,Last_season 3P Jumpshots assisted%
0,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c3425dd,3,2,1,636a5f91531336e79c3425b4,18:50,1130,ThreeMake,7-1,Three,1,5.0,20.7,37.7,25.26,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.035714,0.035714,1.000000,Jumpshot,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,1.071429,3.214286,0.333333,0.000000,0.000000,NaN,0.000000,0.000000,NaN,28.0,0.933333
1,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe45,636a5fa6531336e79c3425fc,7,12,1,636a5f91531336e79c3425b7,13:6,786,ThreeMiss,7-2,Three,0,33.5,31.0,108.3,27.14,True,False,False,False,Fal

In [109]:
cols_to_drop = list()

for time in ['L1','L3','L5','L10','TOT','Last_season']:
    for shot_type in ['Jumpshot','Pullup','Step Back']:
        reduced_3P_df[f"{time} Shot type made"] = np.where(reduced_3P_df['Shot type']==shot_type, reduced_3P_df[f"{time} 3P {shot_type}s made"], reduced_3P_df[f"{time} Shot type made"])
        cols_to_drop.append(f"{time} 3P {shot_type}s made")
        reduced_3P_df[f"{time} Shot type attempted"] = np.where(reduced_3P_df['Shot type']==shot_type, reduced_3P_df[f"{time} 3P {shot_type}s attempted"], reduced_3P_df[f"{time} Shot type attempted"])
        cols_to_drop.append(f"{time} 3P {shot_type}s attempted")
        reduced_3P_df[f"{time} Shot type perc%"] = np.where(reduced_3P_df['Shot type']==shot_type, reduced_3P_df[f"{time} {shot_type}s 3P%"], reduced_3P_df[f"{time} Shot type perc%"])
        cols_to_drop.append(f"{time} {shot_type}s 3P%")

reduced_3P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,Shot type,L1 Shot type made,L1 Shot type attempted,L1 Shot type perc%,L3 Shot type made,L3 Shot type attempted,L3 Shot type perc%,L5 Shot type made,L5 Shot type attempted,L5 Shot type perc%,L10 Shot type made,L10 Shot type attempted,L10 Shot type perc%,TOT Shot type made,TOT Shot type attempted,TOT Shot type perc%,Last_season Shot type made,Last_season Shot type attempted,Last_season Shot type perc%,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots made,L1 3P Jumpshots attempted,L1 Jumpshots 3P%,L1 3P Pullups made,L1 3P Pullups attempted,L1 Pullups 3P%,L1 3P Step Backs made,L1 3P Step Backs attempted,L1 Step Backs 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots made,L3 3P Jumpshots attempted,L3 Jumpshots 3P%,L3 3P Pullups made,L3 3P Pullups attempted,L3 Pullups 3P%,L3 3P Step Backs made,L3 3P Step Backs attempted,L3 Step Backs 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots made,L5 3P Jumpshots attempted,L5 Jumpshots 3P%,L5 3P Pullups made,L5 3P Pullups attempted,L5 Pullups 3P%,L5 3P Step Backs made,L5 3P Step Backs attempted,L5 Step Backs 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots made,L10 3P Jumpshots attempted,L10 Jumpshots 3P%,L10 3P Pullups made,L10 3P Pullups attempted,L10 Pullups 3P%,L10 3P Step Backs made,L10 3P Step Backs attempted,L10 Step Backs 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots made,TOT 3P Jumpshots attempted,TOT Jumpshots 3P%,TOT 3P Pullups made,TOT 3P Pullups attempted,TOT Pullups 3P%,TOT 3P Step Backs made,TOT 3P Step Backs attempted,TOT Step Backs 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots made,Last_season 3P Jumpshots attempted,Last_season Jumpshots 3P%,Last_season 3P Pullups made,Last_season 3P Pullups attempted,Last_season Pullups 3P%,Last_season 3P Step Backs made,Last_season 3P Step Backs attempted,Last_season Step Backs 3P%,Last_season 3P Jumpshots assisted on,Last_season 3P Jumpshots assisted%
0,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c3425dd,3,2,1,636a5f91531336e79c3425b4,18:50,1130,ThreeMake,7-1,Three,1,5.0,20.7,37.7,25.26,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.035714,0.035714,1.000000,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071429,3.214286,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,1.071429,3.214286,0.333333,0.000000,0.000000,NaN,0.000000,0.000000,NaN,28.0,0.933333
1,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe45,636a5fa6531336e79c3425fc,7,12,1,636a5f91531336e79c3425b7,13:6,786,ThreeMiss,7-2,Three,0,33.5,31.0

In [110]:
reduced_3P_df = reduced_3P_df.drop(columns=cols_to_drop)
reduced_3P_df

,game_id,team_id,season_id,player_id,shot_id,score,score_opp,Period,segment_id,Time,Time_Seconds,Stat,Zone,Type,Result,x_coordinate,y_coordinate,Angle,Distance,Jumpshot,Assisted,Red Zone,Floater,Second Chance,Layup,Paint,Drive,Blocked,ATO,Off Turnover,Off Steal,Hook Shot,Fastbreak,Pullup,And1,Turnaround,Dunk,Step Back,D1,games played since,played,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,Shot type,L1 Shot type made,L1 Shot type attempted,L1 Shot type perc%,L3 Shot type made,L3 Shot type attempted,L3 Shot type perc%,L5 Shot type made,L5 Shot type attempted,L5 Shot type perc%,L10 Shot type made,L10 Shot type attempted,L10 Shot type perc%,TOT Shot type made,TOT Shot type attempted,TOT Shot type perc%,Last_season Shot type made,Last_season Shot type attempted,Last_season Shot type perc%,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots assisted on,Last_season 3P Jumpshots assisted%
0,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c3425dd,3,2,1,636a5f91531336e79c3425b4,18:50,1130,ThreeMake,7-1,Three,1,5.0,20.7,37.7,25.26,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.035714,0.035714,1.000000,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071429,3.214286,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,28.0,0.933333
1,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe45,636a5fa6531336e79c3425fc,7,12,1,636a5f91531336e79c3425b7,13:6,786,ThreeMiss,7-2,Three,0,33.5,31.0,108.3,27.14,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.000000,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,3.000000,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.000000,3.000000,0.333333,1.0,1.000000
2,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c342610,11,17,1,636a5f91531336e79c3425b9,10:51,651,ThreeMiss,6-3,Three,0,41.5,23.5,132.1,24.60,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,29.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.321429,0.500000,0.642857,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071429,3.214286,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,28.0,0.933333
3,636a5fc5531336e79c342993,5e875e684dc25ebee5af6797,63502c804e970ea2c1102b09,618d6f44ed05764db467fe48,636a5fa6531336e79c342616,11,19,1,636a5f91531336e79c3425ba,10:6,606,ThreeMiss,7-2,Three,0,26.0,32.0,92.1,26.73,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [111]:
condensed_per_game_3P_features = list(range(-73,0))
condensed_per_game_3P_features.insert(0,38)
condensed_per_game_3P_features.insert(0,4)

In [112]:
reduced_3P_df.iloc[:,condensed_per_game_3P_features].to_pickle("Condensed_3P_features_df.pickle")
reduced_3P_df.iloc[:,condensed_per_game_3P_features]

,shot_id,D1,L1 Zone made,L1 Zone attempted,L1 Zone perc%,L3 Zone made,L3 Zone attempted,L3 Zone perc%,L5 Zone made,L5 Zone attempted,L5 Zone perc%,L10 Zone made,L10 Zone attempted,L10 Zone perc%,TOT Zone made,TOT Zone attempted,TOT Zone perc%,Last_season Zone made,Last_season Zone attempted,Last_season Zone perc%,Shot type,L1 Shot type made,L1 Shot type attempted,L1 Shot type perc%,L3 Shot type made,L3 Shot type attempted,L3 Shot type perc%,L5 Shot type made,L5 Shot type attempted,L5 Shot type perc%,L10 Shot type made,L10 Shot type attempted,L10 Shot type perc%,TOT Shot type made,TOT Shot type attempted,TOT Shot type perc%,Last_season Shot type made,Last_season Shot type attempted,Last_season Shot type perc%,L1 played,L1 3PM,L1 3PA,L1 3P%,L1 3P Jumpshots assisted on,L1 3P Jumpshots assisted%,L3 played,L3 3PM,L3 3PA,L3 3P%,L3 3P Jumpshots assisted on,L3 3P Jumpshots assisted%,L5 played,L5 3PM,L5 3PA,L5 3P%,L5 3P Jumpshots assisted on,L5 3P Jumpshots assisted%,L10 played,L10 3PM,L10 3PA,L10 3P%,L10 3P Jumpshots assisted on,L10 3P Jumpshots assisted%,TOT played,TOT 3PM,TOT 3PA,TOT 3P%,TOT 3P Jumpshots assisted on,TOT 3P Jumpshots assisted%,Last_season played,Last_season 3PM,Last_season 3PA,Last_season 3P%,Last_season 3P Jumpshots assisted on,Last_season 3P Jumpshots assisted%
0,636a5fa6531336e79c3425dd,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.035714,0.035714,1.000000,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071429,3.214286,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,28.0,0.933333
1,636a5fa6531336e79c3425fc,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.000000,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,3.000000,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.000000,3.000000,0.333333,1.0,1.000000
2,636a5fa6531336e79c342610,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.321429,0.500000,0.642857,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071429,3.214286,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,28.0,0.933333
3,636a5fa6531336e79c342616,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071429,0.250000,0.285714,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071429,3.214286,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,28.0,0.933333
4,636a5fa6531336e79c34261a,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.321429,0.892857,0.360000,Jumpshot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.071429,3.214286,0.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,1.071429,3.214286,0.333333,28.0,0.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261966,648b3a7b98debb5769861cbf,True,0.0,0.0,NaN,0.0,0.0,NaN,0.8,1.4,0.571429,0.5,1.4,0.357143,0.310345,0.931034,0.333333,0.290323,1.064516,0.272727,Jumpshot,1.0,4.0,0.25,0.333333,2.666667,0.125,1.0,3.6,0.277778,1.2,4.5,0.266667,1.103448,4.206897,0.262295,1.483871,5.161290,0.287500,1.0,1.0,4.0,0.25,0.0,0.0,3.0,0.333333,2.666667,0.125,0.0,0.000000,5.0,1.0,3.6,0.277778,1.0,0.200000,10.0,1.3,4.6,0.282609,1.0,0.076923,29.0,1.241379,4.37931,0.283465,14.0,0.388889,31.0,1.580645,5.354839,0.295181,21.0